#problems
1. The transcript data we have is a bit less to procede with extracting proper topics because all the models expect a bit more data.

---
#solutions
1. Using a countvectorizer to remove stop words(default) and mentioning it in the model declaration seems to have drastically improved the suggestions by the model.



In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!pip install bertopic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76 kB 2.6 MB/s 
     |████████████████████████████████| 85 kB 3.4 MB/s 
     |████████████████████████████████| 88 kB 7.4 MB/s 
     |████████████████████████████████| 5.2 MB 53.3 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 4.7 MB 44.2 MB/s 
     |████████████████████████████████| 1.3 MB 47.0 MB/s 
     |████████████████████████████████| 101 kB 11.8 MB/s 
     |████████████████████████████████| 636 kB 56.4 MB/s 
     |████████████████████████████████| 6.6 MB 42.6 MB/s 
     |████████████████████████████████| 1.1 MB 56.0 MB/s 
  Created wheel for hdbscan: filename=hdbscan-0.8.28-cp37-cp37m-linux_x86_64.whl size=2340327 sha256=29d6d958d798f41719b8f21ac0b8d6fec3c2c039496af4b719cb22a8f9f5f36d
  Stored in directory: /root/.cache

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from bertopic import BERTopic
from sklearn.datasets import fetch_20newsgroups
import pandas as pd

In [ ]:
%cd /content/drive/MyDrive/Topic_Modelling/

/content/drive/MyDrive/Topic_Modelling


# 1] Using Wells Fargo Transcripts

In [ ]:
with open("Wells_Fargo_transcript.txt","r") as f:
  string = f.read()
print(string)

Operator
Welcome, and thank you for joining the Wells Fargo Second Quarter 2022 Earnings Conference Call. All lines have been placed on mute to prevent any background noise. After the speakers' remarks, there will be a question-and-answer session. [Operator Instructions] Please note that today's call is being recorded. I would now like to turn the call over to your host, John Campbell, Director of Investor Relations. Sir, you may begin the conference.

John Campbell
Good morning. Thank you for joining our call today where our CEO, Charlie Scharf; and our CFO, Mike Santomassimo, will discuss second quarter results and answer your questions. This call is being recorded.

Before we get started, I would like to remind you that our second quarter earnings materials, including the release, financial supplement, and presentation deck are available on our website at wellsfargo.com. I'd also like to caution you that we may make forward-looking statements during today's call that are subject to 

In [ ]:
print(len(string))

68243


In [ ]:
clean_string = []

for s in string.split("\n"):
  if s.isspace() or s == "":
    pass
  else:
    clean_string.append(s)
print(clean_string)
print(len(clean_string))

['Operator', "Welcome, and thank you for joining the Wells Fargo Second Quarter 2022 Earnings Conference Call. All lines have been placed on mute to prevent any background noise. After the speakers' remarks, there will be a question-and-answer session. [Operator Instructions] Please note that today's call is being recorded. I would now like to turn the call over to your host, John Campbell, Director of Investor Relations. Sir, you may begin the conference.", 'John Campbell', 'Good morning. Thank you for joining our call today where our CEO, Charlie Scharf; and our CFO, Mike Santomassimo, will discuss second quarter results and answer your questions. This call is being recorded.', "Before we get started, I would like to remind you that our second quarter earnings materials, including the release, financial supplement, and presentation deck are available on our website at wellsfargo.com. I'd also like to caution you that we may make forward-looking statements during today's call that are

In [ ]:
topic_model_wells_fargo = BERTopic()
topics_custom, probs_custom = topic_model_wells_fargo.fit_transform(clean_string)

In [ ]:
topic_model_wells_fargo.get_topic_info()

,Topic,Count,Name
0,-1,15,-1_juneja_vivek_poonawala_ebrahim
1,0,208,0_the_and_to_of
2,1,39,1_thanks_good_thank_morning
3,2,24,2_charlie_scharf_this_yeah
4,3,21,3_santomassimo_mike__
5,4,19,4_betsy_graseck_question_comes
6,5,15,5_operator_jefferies_questionandanswer_begin
7,6,13,6_siefers_scott_oconnor_matt
8,7,12,7_john_pancari_mcdonald_chubak
9,8,11,8_open_line_your_is


In [ ]:
topic_model_wells_fargo.get_topic(1)

[('thanks', 0.14448749595668992),
 ('good', 0.14364405335637584),
 ('thank', 0.1342885171474543),
 ('morning', 0.1301342104777642),
 ('yes', 0.12162276551950582),
 ('you', 0.11977603850391365),
 ('okay', 0.11943999710606434),
 ('hey', 0.10380178868195887),
 ('all', 0.08623054675319095),
 ('great', 0.0698036853692964)]

In [ ]:
topic_model_wells_fargo.visualize_barchart()

In [ ]:
topic_model_wells_fargo.visualize_topics()

## Using count vectorizer

In [ ]:
with open("Wells_Fargo_names.txt","r") as f:
  name_string = f.read()

In [ ]:
name_string

"John Campbell\tDirector Investor Relations\nCharlie Scharf\tChief Executive Officer\nMike Santomassimo\tChief Financial Officer\nKen Usdin\tJefferies\nJohn McDonald\tAutonomous Research\nSteven Chubak\tWolfe Research\nScott Siefers\tPiper Sandler\nJohn Pancari\tEvercore ISI\nErika Najarian\tUBS\nEbrahim Poonawala\tBank of America\nGerard Cassidy\tRBC Capital Markets\nBetsy Graseck\tMorgan Stanley\nVivek Juneja\tJPMorgan\nMatt O'Connor\tDeutsche Bank"

In [ ]:
cols = ["name","position"]
name_df = pd.read_csv("Wells_Fargo_names.txt",sep="\t",names=cols)

In [ ]:
#trying a new clean function
#since before each paragraph a persons name is mentioned so using that as a reference point
chat_list = []
temp_name = ""
temp_dict = {}
user_names = list(name_df.name)+["Operator"]
for i in clean_string:
  if i in user_names and i!="":
    temp_name = i
    if i not in temp_dict:  
      temp_dict[i]= []
  else:
    if len(i)>30:
      temp_dict[temp_name].append(i)

In [ ]:
temp_dict["Charlie Scharf"]

["Thanks very much, John. Good morning. I'll make some brief comments about our second quarter results, the operating environment, and update you on our priorities. I'll then turn the call over to Mike to review second quarter results in more detail before we take your questions.",
 'Let me start with some second quarter highlights. We earned $3.1 billion in the second quarter, our results included a $576 million impairment of equity securities, predominantly in our venture capital business due to market conditions. Revenue declined as growth in net interest income driven by rising interest rates and higher loan balances was more than offset by lower non-interest income as market conditions negatively impacted our venture capital, mortgage banking, investment banking, and wealth management advisory businesses.',
 'We continued to execute on our efficiency initiatives and our expenses declined from a year ago even with inflationary pressures and higher operating losses. We had broad-bas

In [ ]:
vectorizer_model = CountVectorizer(stop_words="english")

In [ ]:
custom_doc = temp_dict["Charlie Scharf"]+temp_dict["John Campbell"]+temp_dict["Mike Santomassimo"]

In [ ]:
topic_model_custom = BERTopic(vectorizer_model=vectorizer_model)
topics_custom, probs_custom = topic_model_custom.fit_transform(custom_doc)

In [ ]:
topic_model_custom.get_topic_info()

,Topic,Count,Name
0,-1,72,-1_year_just_think_little
1,0,28,0_business_capabilities_wells_fargo
2,1,25,1_quarter_ago_year_higher
3,2,24,2_think_rate_going_hard
4,3,13,3_john_questions_yeah_results
5,4,13,4_think_just_point_certainly


In [ ]:
topic_model_custom.visualize_barchart()

In [ ]:
topic_model_custom.get_topic(1)

[('quarter', 0.0774410299037824),
 ('ago', 0.07656883530508447),
 ('year', 0.07549051903637481),
 ('higher', 0.07020041626901709),
 ('loan', 0.06385935707721643),
 ('income', 0.056157430701528156),
 ('driven', 0.053805863578273785),
 ('net', 0.053428333245728495),
 ('increased', 0.050826682295523506),
 ('growth', 0.047290467959181605)]

In [ ]:
topic_model_custom.visualize_topics()

# Part A: Using transformers on Wells Fargo dataset

### 1) roberta-base

In [ ]:
!python -m pip install bertopic[flair]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from flair.embeddings import TransformerDocumentEmbeddings

In [ ]:
roberta = TransformerDocumentEmbeddings('roberta-base')
topic_model_wells_fargo = BERTopic(embedding_model=roberta,vectorizer_model=vectorizer_model)

In [ ]:
custom_doc = temp_dict["John Campbell"]+temp_dict["Mike Santomassimo"]+temp_dict["Charlie Scharf"]

In [ ]:
topics_custom, probs_custom = topic_model_wells_fargo.fit_transform(clean_string)

In [ ]:
topic_model_wells_fargo.get_topic_info()

,Topic,Count,Name
0,-1,10,-1_open_line_sir_
1,0,156,0_think_just_ll_look
2,1,69,1_year_quarter_ago_higher
3,2,35,2_scharf_charlie_siefers_scott
4,3,29,3_operator_graseck_betsy_juneja
5,4,25,4_just_nii_guys_wondering
6,5,21,5_santomassimo_mike__
7,6,18,6_question_thank_comes_today
8,7,14,7_john_mcdonald_pancari_connor


In [ ]:
topic_model_wells_fargo.get_topic(1)

[('year', 0.053093392325003985),
 ('quarter', 0.04961021632410124),
 ('ago', 0.04817370116319052),
 ('higher', 0.03919521739292028),
 ('driven', 0.03439606959653404),
 ('increased', 0.03359647797823891),
 ('loan', 0.03156893365483556),
 ('rates', 0.031103091592035986),
 ('second', 0.031103091592035986),
 ('income', 0.03007937887348469)]

In [ ]:
topic_model_wells_fargo.visualize_barchart()

In [ ]:
topic_model_wells_fargo.visualize_topics()

### 2) finbert

In [ ]:
#using finbert
finbert = TransformerDocumentEmbeddings('ProsusAI/finbert')
topic_model_wells_fargo = BERTopic(embedding_model=finbert,vectorizer_model=vectorizer_model)

In [ ]:
custom_doc = temp_dict["John Campbell"]+temp_dict["Mike Santomassimo"]+temp_dict["Charlie Scharf"]

In [ ]:
topics_custom, probs_custom = topic_model_wells_fargo.fit_transform(custom_doc)

In [ ]:
topic_model_wells_fargo.get_topic_info()

,Topic,Count,Name
0,-1,3,-1_originations_category_estate_say
1,0,117,0_think_just_ll_look
2,1,29,1_year_ago_higher_increased
3,2,26,2_quarter_year_second_declined


In [ ]:
topic_model_wells_fargo.get_topic(1)

[('year', 0.09469813390340365),
 ('ago', 0.08243298984059556),
 ('higher', 0.0701734347204449),
 ('increased', 0.06296031556464708),
 ('loan', 0.05560938087782007),
 ('increase', 0.0536272462262859),
 ('driven', 0.05328104224609513),
 ('income', 0.051923284837833435),
 ('rates', 0.049067100774547116),
 ('commercial', 0.04758041631374204)]

In [ ]:
topic_model_wells_fargo.visualize_barchart()

In [ ]:
# topic_model_wells_fargo.visualize_topics()

# *] Using Apple Transcripts

In [ ]:
with open("Apple_transcript.txt","r") as f:
  string = f.read().split("\n")
  clean_string = []
  for s in string:
    if s.isspace() or s == "":
      pass
    else:
      clean_string.append(s)
print(clean_string)
print(len(clean_string))

['Operator', "Good day, and welcome to the Apple Q3 FY 2022 Earnings Conference Call. Today's call is being recorded. At this time, for opening remarks and introductions, I would like to turn the call over to Tejas Gala, Director of Investor Relations and Corporate Finance. Please go ahead.", 'Tejas Gala', 'Thank you. Good afternoon, and thank you for joining us.', "Speaking first today is Apple's CEO, Tim Cook; and he'll be followed by CFO, Luca Maestri. After that, we'll open the call to questions from analysts. Please note that some of the information you'll hear during the discussion today will consist of forward-looking statements, including, without limitation, those regarding revenue, gross margin, operating expenses, other income and expense, taxes, capital allocation and future business outlook, including the potential impact of COVID-19 on the company's business and results of operation. These statements involve risks and uncertainties that may cause actual results or trends 

In [ ]:
topic_model_apple = BERTopic()
topics_custom, probs_custom = topic_model_apple.fit_transform(clean_string)

In [ ]:
topic_model_apple.get_topic_info()

,Topic,Count,Name
0,0,148,0_the_and_of_to
1,1,49,1_question_take_next_maestri
2,2,16,2_timothy_cook__
3,3,14,3_operator_instructions_first_question


In [ ]:
topic_model_apple.get_topic(1)

[('question', 0.19575421593237502),
 ('take', 0.19356462125773308),
 ('next', 0.19053199190092504),
 ('maestri', 0.16251494241339262),
 ('well', 0.16156873041985648),
 ('luca', 0.15168646878204062),
 ('from', 0.1421889120337387),
 ('with', 0.12905608190784587),
 ('our', 0.11627632260778785),
 ('suva', 0.10457853573424342)]

In [ ]:
topic_model_apple.visualize_topics()

In [ ]:
topic_model_apple.visualize_barchart()

## Using count vectorizer

In [ ]:
cols = ["name","position"]
name_df = pd.read_csv("Apple_names.txt",sep="\t",names=cols)

chat_list = []
temp_name = ""
temp_dict = {}
user_names = list(name_df.name)+["Operator"]
for i in clean_string:
  if i in user_names and i!="":
    temp_name = i
    if i not in temp_dict:  
      temp_dict[i]= []
  else:
    if len(i)>30:
      temp_dict[temp_name].append(i)

In [ ]:
custom_doc = temp_dict['Tejas Gala']+ temp_dict["Timothy Cook"] +temp_dict["Luca Maestri"]

In [ ]:
topic_model_custom_apple = BERTopic(vectorizer_model=vectorizer_model)
topics_custom, probs_custom = topic_model_custom_apple.fit_transform(custom_doc)

In [ ]:
topic_model_custom_apple.get_topic_info()

,Topic,Count,Name
0,-1,52,-1_quarter_june_world_mentioned
1,0,20,0_new_customers_apple_iphone
2,1,15,1_billion_quarter_revenue_june
3,2,14,2_impact_exchange_foreign_basis
4,3,12,3_quarter_think_guidance_services


In [ ]:
topic_model_custom_apple.get_topic(1)

[('billion', 0.11407912158738552),
 ('quarter', 0.1125213844225215),
 ('revenue', 0.10865489823373531),
 ('june', 0.10419032951722554),
 ('records', 0.08698427545734187),
 ('record', 0.0848539551894464),
 ('set', 0.06958742036587351),
 ('services', 0.06633305969906429),
 ('alltime', 0.0509881178792257),
 ('cash', 0.04888739611648869)]

In [ ]:
topic_model_custom_apple.visualize_barchart()

In [ ]:
topic_model_custom_apple.visualize_topics()

In [ ]:
custom_doc = temp_dict["Timothy Cook"]
topic_model_custom_apple = BERTopic(vectorizer_model=vectorizer_model)
topic_model_custom, probs_custom = topic_model_custom_apple.fit_transform(custom_doc)

In [ ]:
topic_model_custom_apple.get_topic_info()

,Topic,Count,Name
0,-1,13,-1_revenue_saw_buy_quarter
1,0,22,0_new_apple_iphone_customers
2,1,18,1_quarter_june_covid_world


In [ ]:
topic_model_custom_apple.visualize_barchart()

In [ ]:
#collective data 
full_conversation = []
for i in temp_dict.values():
  full_conversation+=i
topic_model_custom_apple = BERTopic(vectorizer_model=vectorizer_model)
topic_model_custom, probs_custom = topic_model_custom_apple.fit_transform(full_conversation)

In [ ]:
topic_model_custom_apple.get_topic_info()

,Topic,Count,Name
0,-1,23,-1_like_market_thank_good
1,0,70,0_quarter_think_business_impact
2,1,56,1_apple_new_quarter_revenue
3,2,11,2_question_operator_instructions_research


In [ ]:
topic_model_custom_apple.get_topic(1)

[('apple', 0.05269999522022413),
 ('new', 0.050882754005733645),
 ('quarter', 0.04585333327318677),
 ('revenue', 0.041579745110405145),
 ('billion', 0.040750092060578155),
 ('june', 0.04060657079335405),
 ('iphone', 0.037530082630260794),
 ('customers', 0.03514574037526219),
 ('record', 0.03372022358731016),
 ('services', 0.02830345152511076)]

In [ ]:
topic_model_custom_apple.visualize_barchart()

In [ ]:
# topic_model_custom_apple.visualize_topics()

# Part A: Using transformers on Apple dataset


In [ ]:
!python -m pip install bertopic[flair]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### 1) roberta-base

In [ ]:
from flair.embeddings import TransformerDocumentEmbeddings

In [ ]:
roberta = TransformerDocumentEmbeddings('roberta-base')
topic_model_apple = BERTopic(embedding_model=roberta,vectorizer_model=vectorizer_model)

In [ ]:
with open("Apple_transcript.txt","r") as f:
  string = f.read().split("\n")
  clean_string = []
  for s in string:
    if s.isspace() or s == "":
      pass
    else:
      clean_string.append(s)

cols = ["name","position"]
name_df = pd.read_csv("Apple_names.txt",sep="\t",names=cols)

chat_list = []
temp_name = ""
temp_dict = {}
user_names = list(name_df.name)+["Operator"]
for i in clean_string:
  if i in user_names and i!="":
    temp_name = i
    if i not in temp_dict:  
      temp_dict[i]= []
  else:
    if len(i)>30:
      temp_dict[temp_name].append(i)

In [ ]:
custom_doc = temp_dict["Timothy Cook"] +temp_dict["Luca Maestri"]

In [ ]:
topics_custom, probs_custom = topic_model_apple.fit_transform(custom_doc)

In [ ]:
topic_model_apple.get_topic_info()

,Topic,Count,Name
0,-1,7,-1_year_quarter_outlook_revenue
1,0,44,0_quarter_expect_developers_different
2,1,33,1_quarter_think_ve_look
3,2,24,2_new_apple_billion_quarter


In [ ]:
topic_model_apple.get_topic(1)

[('quarter', 0.07231699626506533),
 ('think', 0.05657845691325456),
 ('ve', 0.04909604181249033),
 ('look', 0.04808105781707793),
 ('impact', 0.04771121017972473),
 ('june', 0.044191967839828365),
 ('business', 0.04337382743611339),
 ('saw', 0.04327295203537014),
 ('mentioned', 0.038464846253662345),
 ('going', 0.0360094303964174)]

In [ ]:
topic_model_apple.visualize_barchart()

In [ ]:
# topic_model_apple.visualize_topics()

### 2) finbert

In [ ]:
#using finbert
finbert = TransformerDocumentEmbeddings('ProsusAI/finbert')
topic_model_apple = BERTopic(embedding_model=finbert,vectorizer_model=vectorizer_model)

In [ ]:
custom_doc = temp_dict["Timothy Cook"]

In [ ]:
topics_custom, probs_custom = topic_model_apple.fit_transform(custom_doc)

In [ ]:
topic_model_apple.get_topic_info()

,Topic,Count,Name
0,0,40,0_apple_new_quarter_customers
1,1,13,1_quarter_june_saw_macroeconomic


In [ ]:
topic_model_apple.get_topic(1)

[('quarter', 0.13414183004166022),
 ('june', 0.09556682523611233),
 ('saw', 0.08756147256272476),
 ('macroeconomic', 0.08471619119801592),
 ('impact', 0.062491006682704166),
 ('covid', 0.062491006682704166),
 ('look', 0.060511565141439946),
 ('home', 0.060511565141439946),
 ('business', 0.058799189661106005),
 ('year', 0.05594327148665919)]

In [ ]:
topic_model_apple.visualize_barchart()

In [ ]:
# topic_model_apple.visualize_topics()

In [ ]:
#collective data 
full_conversation = []
for i in temp_dict.values():
  full_conversation+=i

In [ ]:
topic_model_apple = BERTopic(embedding_model=finbert,vectorizer_model=vectorizer_model)

In [ ]:
topics_custom, probs_custom = topic_model_apple.fit_transform(full_conversation)

In [ ]:
topic_model_apple.get_topic_info()

,Topic,Count,Name
0,-1,23,-1_quarter_billion_margin_gross
1,0,67,0_think_apple_new_iphone
2,1,35,1_impact_quarter_year_sort
3,2,20,2_quarter_june_revenue_year
4,3,15,3_question_ll_operator_today


In [ ]:
topic_model_apple.get_topic(1)

[('impact', 0.07032994913200458),
 ('quarter', 0.055530054393738745),
 ('year', 0.04881148190093715),
 ('sort', 0.04498886906131903),
 ('saw', 0.04498886906131903),
 ('june', 0.042303284314145526),
 ('business', 0.04031390541983836),
 ('mentioned', 0.03980475893646674),
 ('just', 0.036958294906235825),
 ('covid', 0.03599109524905523)]

In [ ]:
topic_model_apple.visualize_barchart()

In [ ]:
topic_model_apple.visualize_topics()

In [ ]:
topic_model_apple = BERTopic(embedding_model=roberta,vectorizer_model=vectorizer_model)
topics_custom, probs_custom = topic_model_apple.fit_transform(full_conversation)

In [ ]:
topic_model_apple.get_topic_info()

,Topic,Count,Name
0,-1,40,-1_ll_question_today_luca
1,0,41,0_product_quarter_developers_number
2,1,34,1_quarter_business_impact_look
3,2,24,2_new_apple_billion_quarter
4,3,21,3_sort_just_maybe_follow


In [ ]:
topic_model_apple.get_topic(1)

[('quarter', 0.07617027572891613),
 ('business', 0.04732430725617796),
 ('impact', 0.046572308959666346),
 ('look', 0.0461313941092364),
 ('think', 0.04604228654058297),
 ('year', 0.045251970483356704),
 ('june', 0.045251970483356704),
 ('ve', 0.04322450779330514),
 ('saw', 0.040218540934035034),
 ('macroeconomic', 0.03787028455922516)]

In [ ]:
topic_model_apple.visualize_barchart()

In [ ]:
# topic_model_apple.visualize_topics()

# 2] Using CISCO Transcripts

In [ ]:
with open("Cisco_transcript.txt","r") as f:
  string = f.read().split("\n")
  clean_string = []
  for s in string:
    if s.isspace() or s == "":
      pass
    else:
      clean_string.append(s)
print(clean_string)
print(len(clean_string))

['Operator', "Welcome to Cisco's Third Quarter Fiscal Year 2022 Financial Results Conference Call. At the request of Cisco, today's conference is being recorded.", 'If you have any objections, you may disconnect.', "Now I would like to introduce Marilyn Mora, Head of Investor Relations. Ma'am, you may begin.", 'Marilyn Mora', "Welcome everyone to Cisco's third quarter fiscal 2022 quarterly earnings conference call. This is Marilyn Mora, Head of Investor Relations, and I'm joined by Chuck Robbins, our Chair and CEO; and Scott Herren, our CFO. By now, you should have seen our earnings press release. A corresponding webcast with slides, including supplemental information, will be made available on our website in the investor relations section following the call. Income statements, full GAAP to non-GAAP reconciliation information, balance sheets, cash flow statements and other financial information can also be found in the financial information section of our investor relations website. Th

In [ ]:
topic_model_cisco = BERTopic()
topics_custom, probs_custom = topic_model_cisco.fit_transform(clean_string)

In [ ]:
topic_model_cisco.get_topic_info()

,Topic,Count,Name
0,-1,7,-1_vogt_david_ferragu_ubs
1,0,196,0_the_of_and_to
2,1,41,1_thanks_thank_tim_ittai
3,2,26,2_herren_scott_harwood_ben
4,3,23,3_okay_so_it_about
5,4,20,4_robbins_chuck__
6,5,17,5_mora_marilyn_maam_introduce
7,6,15,6_thank_you_thanks_okay
8,7,14,7_operator___
9,8,14,8_may_ahead_go_sir


In [ ]:
topic_model_cisco.visualize_topics()

In [ ]:
topic_model_cisco.visualize_barchart()

In [ ]:
topic_model_cisco.get_topic(1)

[('thanks', 0.164765700964103),
 ('thank', 0.15558395042224893),
 ('tim', 0.13524530618572575),
 ('ittai', 0.13524530618572575),
 ('samik', 0.13524530618572575),
 ('tal', 0.13047820327790827),
 ('you', 0.11725244306178115),
 ('liani', 0.11286930875177738),
 ('meta', 0.11286930875177738),
 ('scott', 0.10984380064273533)]

## Using count vectorizer

In [ ]:
cols = ["name","position"]
name_df = pd.read_csv("Cisco_names.txt",sep="\t",names=cols)

chat_list = []
temp_name = ""
temp_dict = {}
user_names = list(name_df.name)+["Operator"]
for i in clean_string:
  if i in user_names and i!="":
    temp_name = i
    if i not in temp_dict:  
      temp_dict[i]= []
  else:
    if len(i)>15:
      temp_dict[temp_name].append(i)

In [ ]:
temp_dict["Chuck Robbins"]

['Thanks, Marilyn, and good afternoon, everyone. When we spoke with you back in February, we entered Q3 in the second half of our fiscal year with optimism despite the supply and component challenges and other headwinds impacting us and many of our peers. Many of those factors that fueled that optimism remain unchanged today.',
 'We continue to see strong demand resulting in record backlog, our business transformation is progressing well and our differentiated innovation across our portfolio is helping our customers embrace and adopt the multiple technology transitions happening.',
 'However, there were two unanticipated events since our last earnings call, which impacted our Q3 revenue performance.',
 'The first is the war in Ukraine. This resulted in us ceasing operations in Russia and Belarus and had a corresponding revenue impact, which Scott will discuss.',
 'The second relates to COVID related lockdowns in China, which began in late March. These lockdowns resulted in an even more

In [ ]:
custom_doc = temp_dict["Chuck Robbins"]+temp_dict["Marilyn Mora"]+temp_dict["Scott Herren"]

In [ ]:
vectorizer_model = CountVectorizer(stop_words="english")

In [ ]:
topic_model_custom = BERTopic(vectorizer_model=vectorizer_model)
topics_custom, probs_custom = topic_model_custom.fit_transform(custom_doc)

In [ ]:
topic_model_custom.get_topic_info()

,Topic,Count,Name
0,0,151,0_quarter_think_revenue_growth
1,1,37,1_question_scott_yes_thanks


In [ ]:
topic_model_custom.visualize_barchart()

In [ ]:
# topic_model_custom.visualize_topics()

In [ ]:
topic_model_custom.get_topic(1)

[('question', 0.3542011135426437),
 ('scott', 0.2394435834953846),
 ('yes', 0.21252066812558623),
 ('thanks', 0.16285373296918715),
 ('thinking', 0.14741540825704258),
 ('thats', 0.138472771434522),
 ('thank', 0.12791406476949568),
 ('want', 0.11753520740161166),
 ('way', 0.11554012670347229),
 ('im', 0.11554012670347229)]

# Part A: Using transformers on CISCO dataset


### 1) roberta-base

In [ ]:
roberta = TransformerDocumentEmbeddings('roberta-base')
vectorizer_model = CountVectorizer(stop_words="english",token_pattern='[a-zA-Z0-9]{3,}' )
topic_model_cisco = BERTopic(embedding_model=roberta,vectorizer_model=vectorizer_model,top_n_words = 10,n_gram_range =(1,2))

In [ ]:
custom_data = temp_dict["Chuck Robbins"] + temp_dict["Marilyn Mora"] + temp_dict["Scott Herren"]

In [ ]:
topics_custom, probs_custom = topic_model_cisco.fit_transform(custom_data)

In [ ]:
topic_model_cisco.get_topic_info()

,Topic,Count,Name
0,-1,76,-1_quarter_year_supply_question
1,0,33,0_think_quarter_going_just
2,1,28,1_yes_let_thanks_great
3,2,20,2_year_billion_total_revenue
4,3,18,3_levels_hardware_lockdowns_guide
5,4,13,4_seeing_million_peak_problem


In [ ]:
topic_model_cisco.get_topic(1)

[('yes', 0.23721040073683508),
 ('let', 0.14176601219795254),
 ('thanks', 0.14176601219795254),
 ('great', 0.1212150722746754),
 ('question', 0.11860520036841754),
 ('scott', 0.10366798122652693),
 ('don', 0.10126907530404204),
 ('way', 0.09908288055363583),
 ('simon', 0.09638213109972216),
 ('answer', 0.0894814929824202)]

In [ ]:
topic_model_cisco.visualize_barchart()

In [ ]:
topic_model_cisco.visualize_topics()

### 2) finbert

In [ ]:
#using finbert
finbert = TransformerDocumentEmbeddings('ProsusAI/finbert')
topic_model_cisco = BERTopic(embedding_model=finbert,vectorizer_model=vectorizer_model)

In [ ]:
topics_custom, probs_custom = topic_model_cisco.fit_transform(custom_data)

In [ ]:
topic_model_cisco.get_topic_info()

,Topic,Count,Name
0,-1,17,-1_question_thanks_questions_today
1,0,130,0_think_revenue_quarter_just
2,1,41,1_year_business_demand_quarter


In [ ]:
topic_model_cisco.visualize_barchart()

In [ ]:
# topic_model_cisco.visualize_topics()

In [ ]:
topic_model_cisco.get_topic(1)

[('year', 0.0645458628093601),
 ('business', 0.05260399865809979),
 ('demand', 0.050754208271132695),
 ('quarter', 0.05034220563296519),
 ('growth', 0.04829857414731088),
 ('customers', 0.0455205213512252),
 ('strong', 0.04382169004406858),
 ('continue', 0.04235329272237923),
 ('innovation', 0.03808997749573752),
 ('deliver', 0.03808997749573752)]

# 3] Using Intuit Transcripts

In [ ]:
with open("Intuit_transcript.txt","r") as f:
  string = f.read().split("\n")

In [ ]:
  clean_string = []
  for s in string:
    if s.isspace() or s == "":
      pass
    else:
      clean_string.append(s)

In [ ]:
topic_model_intuit = BERTopic()
topics_custom, probs_custom = topic_model_intuit.fit_transform(clean_string)

In [ ]:
topic_model_intuit.get_topic_info()

,Topic,Count,Name
0,0,208,0_the_and_of_to
1,1,61,1_thank_you_very_welcome
2,2,39,2_goodarzi_sasan_evening_good
3,3,39,3_comes_next_from_question
4,4,16,4_operator_mizuho_instructions_siti
5,5,12,5_michelle_clatterbuck_for_maybe


In [ ]:
topic_model_intuit.visualize_topics()

In [ ]:
topic_model_intuit.visualize_barchart()

## Using count vectorizer

In [ ]:
cols = ["name","position"]
name_df = pd.read_csv("Intuit_names.txt",sep="\t",names=cols)

chat_list = []
temp_name = ""
temp_dict = {}
user_names = list(name_df.name)+["Operator"]
for i in clean_string:
  if i in user_names and i!="":
    temp_name = i
    if i not in temp_dict:  
      temp_dict[i]= []
  else:
    if len(i)>15:
      temp_dict[temp_name].append(i)

In [ ]:
temp_dict["Sasan Goodarzi"]

['Thanks, Kim and thanks to all of you for joining us today. I am proud of our continued momentum as we execute on our strategy to be the global AI-driven expert platform powering the prosperity of consumers and small businesses.',
 'We have a nearly $300 billion addressable market driven by tailwinds that include shifts to virtual solutions, acceleration to online and omnichannel capabilities and digital money offerings. This, combined with the team’s execution, is contributing to the strength of our performance.',
 'Third quarter revenue grew 35%, including 6 points from the addition of Mailchimp. It was another strong quarter for our Small Business and Self-Employed Group with revenue up 42%, 20% organically. Credit Karma posted another quarter with revenue at an all-time high, up 48%. And I am proud of how the team executed in the Consumer Group in another unusual tax season.',
 'We are confident in our business trajectory and are raising our total Intuit revenue and non-GAAP earni

In [ ]:
vectorizer_model = CountVectorizer(stop_words="english")

In [ ]:
custom_doc = temp_dict["Sasan Goodarzi"]+temp_dict["Kim Watkins"]+temp_dict["Michelle Clatterbuck"]

In [ ]:
topic_model_custom = BERTopic(vectorizer_model=vectorizer_model)
topics_custom, probs_custom = topic_model_custom.fit_transform(custom_doc)

In [ ]:
topic_model_custom.get_topic_info()

,Topic,Count,Name
0,-1,5,-1_unidentified_analyst__
1,0,169,0_platform_growth_year_customers
2,1,23,1_welcome_thank_yes_question


In [ ]:
topic_model_custom.get_topic(1)

[('welcome', 0.4651366180261114),
 ('thank', 0.36951370860807325),
 ('yes', 0.2829315838220342),
 ('question', 0.23219667607228883),
 ('michelle', 0.21547144058142578),
 ('questions', 0.21547144058142578),
 ('youre', 0.21547144058142578),
 ('thanks', 0.20402513330491842),
 ('alright', 0.17286246754838072),
 ('keith', 0.15441931746106694)]

In [ ]:
topic_model_custom.visualize_barchart()

In [ ]:
# topic_model_custom.visualize_topics()

# Part A: Using transformers on Intuit dataset



### 1) roberta-base

In [ ]:
from flair.embeddings import TransformerDocumentEmbeddings

roberta = TransformerDocumentEmbeddings('roberta-base')
vectorizer_model = CountVectorizer(stop_words="english",token_pattern='[a-zA-Z0-9]{3,}' )
topic_model_intuit = BERTopic(embedding_model=roberta,vectorizer_model=vectorizer_model,top_n_words = 10,n_gram_range =(1,2))

In [ ]:
custom_data = temp_dict["Sasan Goodarzi"] + temp_dict["Michelle Clatterbuck"] + temp_dict["Kim Watkins"]

In [ ]:
topics_custom, probs_custom = topic_model_intuit.fit_transform(custom_data)

In [ ]:
topic_model_intuit.get_topic_info()

,Topic,Count,Name
0,-1,48,-1_gaap_intuit_non_share
1,0,42,0_platform_just_look_sort
2,1,40,1_revenue_year_growth_irs
3,2,27,2_continue_principles_revenue_platform
4,3,26,3_welcome_thank_yes_question
5,4,14,4_think_really_sure_benefits


In [ ]:
topic_model_intuit.get_topic(1)

[('revenue', 0.07519181166317451),
 ('year', 0.06810343605712141),
 ('growth', 0.06741088228749834),
 ('irs', 0.04809750975413178),
 ('expect', 0.047445915686427066),
 ('customers', 0.04545788109691325),
 ('returns', 0.041266611751283225),
 ('quarter', 0.04068604257553743),
 ('grew', 0.038348333667370164),
 ('credit', 0.03722409561150605)]

In [ ]:
topic_model_intuit.visualize_barchart()

In [ ]:
topic_model_intuit.visualize_topics()

### 2) finbert

In [ ]:
#using finbert
finbert = TransformerDocumentEmbeddings('ProsusAI/finbert')
topic_model_intuit = BERTopic(embedding_model=finbert,vectorizer_model=vectorizer_model)

In [ ]:
topics_custom, probs_custom = topic_model_intuit.fit_transform(custom_data)

In [ ]:
topic_model_intuit.get_topic_info()

,Topic,Count,Name
0,0,182,0_year_platform_growth_customers
1,1,15,1_welcome_thank_yes_thanks


In [ ]:
topic_model_intuit.get_topic(1)

[('welcome', 0.9871253582612076),
 ('thank', 0.7883756212214635),
 ('yes', 0.4859665905646103),
 ('thanks', 0.4323110844126661),
 ('questions', 0.30329343971040484),
 ('question', 0.24878248005082162),
 ('hey', 0.19872926882503447),
 ('wonderful', 0.19872926882503447),
 ('taking', 0.19872926882503447),
 ('safe', 0.19872926882503447)]

In [ ]:
# topic_model_intuit.visualize_topics()

In [ ]:
topic_model_intuit.visualize_barchart()

# 4] Using UHG Transcripts

In [ ]:
with open("UHG_transcript.txt","r") as f:
  string = f.read().split("\n")

In [ ]:
clean_string = []
for s in string:
  if s.isspace() or s == "":
    pass
  else:
    clean_string.append(s)  
print(clean_string)
print(len(clean_string))

['Operator', "Good morning, and welcome to the UnitedHealth Group’s Second Quarter 2022 Earnings Conference Call. A question-and-answer session will follow UnitedHealth Group's prepared remarks.", 'As a reminder, this call is being recorded.', "Here are some important introductory information. This call contains forward-looking statements under the U.S. federal securities laws. These statements are subject to risks and uncertainties that could cause actual results to differ materially from historical experience or present expectations. A description of some of these risks and uncertainties can be found in the reports that we file with the Securities and Exchange Commission, including the cautionary statements included in our current and periodic filings. This call will also reference non-GAAP amounts. A reconciliation of the non-GAAP to GAAP amounts is available on the Financial and Earnings Reports section of the company's Investor Relations page at www.unitedhealthgroup.com. Informat

In [ ]:
topic_model_UHG= BERTopic()
topics_custom, probs_custom = topic_model_UHG.fit_transform(clean_string)

In [ ]:
topic_model_UHG.get_topic_info()

,Topic,Count,Name
0,0,140,0_the_and_to_of
1,1,66,1_next_question_thanks_well
2,2,26,2_witty_andrew_rich_nathan
3,3,14,3_operator_final_please_question


In [ ]:
topic_model_UHG.get_topic(1)

[('next', 0.20731330702636802),
 ('question', 0.20130048239941598),
 ('thanks', 0.1428006433105415),
 ('well', 0.13715799575194976),
 ('go', 0.1288624916404463),
 ('john', 0.1260331015959894),
 ('please', 0.1157502297727247),
 ('much', 0.11517405945909334),
 ('with', 0.10868372540995706),
 ('thank', 0.08527071756219329)]

In [ ]:
topic_model_UHG.visualize_topics()

In [ ]:
topic_model_UHG.visualize_barchart()

## Using count vectorizer

In [ ]:
cols = ["name","position"]
name_df = pd.read_csv("UHG_names.txt",sep="\t",names=cols)

chat_list = []
temp_name = ""
temp_dict = {}
user_names = list(name_df.name)+["Operator"]
for i in clean_string:
  if i in user_names and i!="":
    temp_name = i
    if i not in temp_dict:  
      temp_dict[i]= []
  else:
    if len(i)>15:
      temp_dict[temp_name].append(i)

In [ ]:
temp_dict["Andrew Witty"]

['Katy, thank you, and good morning, everybody, and thank you for joining us today. UnitedHealth Group ended the second half of the year with sustained momentum as we execute on our objective to serve more people more effectively with connected high-quality care.',
 "For that, I want to thank our 360,000 colleagues. It's their unwavering commitment to our mission and their hard work and support of the people we serve that makes all of this possible.",
 "As a result of the strong performance at both Optum and UnitedHealthcare, we are increasing our adjusted earnings per share outlook for the year to a range of $21.40 to $21.90 per share. Comprehensive value-based care is a central theme of our growth strategy, helping more patients and care providers transition from traditional fee-for-service to a value-based orientation. We aim to drive better and more consistent care outcomes at lower overall cost, often for people who are among society's most vulnerable with multiple chronic conditi

In [ ]:
vectorizer_model = CountVectorizer(stop_words="english")
print(vectorizer_model)

CountVectorizer(stop_words='english')


In [ ]:
custom_doc = temp_dict["Andrew Witty"]+temp_dict["Dirk McMahon"]+temp_dict["John Rex"]

In [ ]:
topic_model_UHG = BERTopic(vectorizer_model=vectorizer_model)
topics_custom, probs_custom = topic_model_UHG.fit_transform(custom_doc)

In [ ]:
topic_model_UHG.get_topic_info()

,Topic,Count,Name
0,-1,10,-1_question_qa_underscore_final
1,0,89,0_care_really_optum_think
2,1,19,1_thanks_question_thank_john


In [ ]:
topic_model_UHG.get_topic(1)

[('thanks', 0.42808046888538254),
 ('question', 0.3465555979737732),
 ('thank', 0.2884703608342235),
 ('john', 0.18857785829706478),
 ('justin', 0.1863173740926968),
 ('ricky', 0.1863173740926968),
 ('dirk', 0.1863173740926968),
 ('appreciate', 0.16382219089716119),
 ('aj', 0.13300211300089887),
 ('kevin', 0.12421158272846454)]

In [ ]:
topic_model_UHG.visualize_barchart()

In [ ]:
# topic_model_UHG.visualize_topics()

# Part A: Using transformers on UHG dataset


### 1) roberta-base

In [ ]:
from flair.embeddings import TransformerDocumentEmbeddings

roberta = TransformerDocumentEmbeddings('roberta-base')
vectorizer_model = CountVectorizer(stop_words="english",token_pattern='[a-zA-Z0-9]{3,}' )
topic_model_UHG = BERTopic(embedding_model=roberta,vectorizer_model=vectorizer_model,top_n_words = 10,n_gram_range =(1,2))

In [ ]:
custom_data = temp_dict["Andrew Witty"] + temp_dict["Dirk McMahon"] + temp_dict["John Rex"] + temp_dict["Brian Thompson"]

In [ ]:
topics_custom, probs_custom = topic_model_UHG.fit_transform(custom_data)

In [ ]:
topic_model_UHG.get_topic_info()

,Topic,Count,Name
0,0,115,0_care_really_think_optum
1,1,12,1_question_let_straight_underscore


In [ ]:
topic_model_UHG.get_topic(1)

[('question', 1.692506208597891),
 ('let', 0.506754018901455),
 ('straight', 0.3458394571892496),
 ('underscore', 0.3458394571892496),
 ('final', 0.3458394571892496),
 ('operator', 0.31286583032937576),
 ('turn', 0.2935913263919724),
 ('hand', 0.2935913263919724),
 ('points', 0.27992551508618935),
 ('key', 0.2693329433723739)]

In [ ]:
# topic_model_UHG.visualize_topics()

In [ ]:
topic_model_UHG.visualize_barchart()

### 2) finbert

In [ ]:
#using finbert
finbert = TransformerDocumentEmbeddings('ProsusAI/finbert')
topic_model_UHG = BERTopic(embedding_model=finbert,vectorizer_model=vectorizer_model)

In [ ]:
topics_custom, probs_custom = topic_model_UHG.fit_transform(custom_data)

In [ ]:
topic_model_UHG.get_topic_info()

,Topic,Count,Name
0,-1,20,-1_care_covid_really_emergency
1,0,52,0_think_really_care_just
2,1,15,1_thanks_thank_question_ricky
3,2,15,2_billion_year_offerings_growth
4,3,13,3_care_consumers_optum_value
5,4,12,4_question_straight_gary_final


In [ ]:
topic_model_UHG.get_topic(1)

[('thanks', 0.4402859288483107),
 ('thank', 0.28772000134952835),
 ('question', 0.21937106277907864),
 ('ricky', 0.1925695611578486),
 ('justin', 0.18172428745408956),
 ('morning', 0.16648510636244457),
 ('great', 0.1557180422268128),
 ('john', 0.13767212841993456),
 ('kevin', 0.12837970743856572),
 ('questions', 0.12114952496939303)]

In [ ]:
topic_model_UHG.visualize_barchart()

In [ ]:
topic_model_UHG.visualize_topics()

# 5] Using Thermofisher Transcripts

In [ ]:
with open("Thermo_Fisher_transcript.txt","r") as f:
  string = f.read().split("\n")
  clean_string = []
  for s in string:
    if s.isspace() or s == "":
      pass
    else:
      clean_string.append(s)
print(clean_string)
print(len(clean_string))

['Operator', 'Good morning, ladies and gentlemen. Welcome to the Thermo Fisher Scientific 2022 Second Quarter Conference Call. My name is Jacquita. I will be your operator for today’s call. [Operator Instructions] I would like to introduce our moderator for the call, Mr. Rafael Tejada, Vice President of Investor Relations. Mr. Tejada, you may begin the call.', 'Rafael Tejada', 'Good morning and thank you for joining us.', 'On the call with me today is Marc Casper, our Chairman, President and Chief Executive Officer; and Stephen Williamson, Senior Vice President and Chief Financial Officer. Please note this call is being webcast live and will be archived on the Investors section of our website, thermofisher.com, under the heading News and Events until August 12, 2022. A copy of the press release of our second quarter 2022 earnings is available in the Investors section of our website under the heading, Financials.', 'So before we begin, let me briefly cover our Safe Harbor statement. Var

In [ ]:
topic_model_thermofisher = BERTopic()
topics_custom, probs_custom = topic_model_thermofisher.fit_transform(clean_string)

In [ ]:
topic_model_thermofisher.get_topic_info()

,Topic,Count,Name
0,-1,11,-1_yes_puneet_souda_morning
1,0,196,0_the_and_in_of
2,1,72,1_thanks_you_thank_call
3,2,33,2_casper_marc_donnelly_patrick
4,3,12,3_operator_question_nephron_instructions


In [ ]:
topic_model_thermofisher.get_topic(1)

[('thanks', 0.12623829197487926),
 ('you', 0.12548177557274318),
 ('thank', 0.0995773775296978),
 ('call', 0.09841047248845002),
 ('stephen', 0.09406749163990615),
 ('question', 0.09093838858459316),
 ('may', 0.09052488866336163),
 ('the', 0.08848783077858594),
 ('for', 0.08609601394930971),
 ('proceed', 0.08505400652868357)]

In [ ]:
topic_model_thermofisher.visualize_barchart()

In [ ]:
topic_model_thermofisher.visualize_topics()

## Using count vectorizer

In [ ]:
cols = ["name","position"]
name_df = pd.read_csv("Thermo_Fisher_names.txt",sep="\t",names=cols)

chat_list = []
temp_name = ""
temp_dict = {}
user_names = list(name_df.name)+["Operator"]
for i in clean_string:
  if i in user_names and i!="":
    temp_name = i
    if i not in temp_dict:  
      temp_dict[i]= []
  else:
    if len(i)>15:
      temp_dict[temp_name].append(i)

In [ ]:
temp_dict["Marc Casper"]

['Thank you, Raf. Good morning, everyone and thanks for joining us today for our second quarter call.',
 'As you saw in our press release, we had another excellent quarter. We delivered outstanding financial performance.',
 'Our core business is performing very well, that strength is broad-based across all our businesses.',
 'As I reflect on the quarter, I am very proud of the team’s great execution and the resulting share gain we saw across our business.',
 'Our ongoing success is propelled by our proven growth strategy and our PPI Business System, which is a differentiator for us and enables operational excellence within the company.',
 'You will see this in our second quarter results and increased outlook for the year.',
 'So, let me first recap the financials.',
 'Our revenue in the quarter grew 18% year-over-year to $10.97 billion.',
 'Our adjusted operating income was $2.61 billion.',
 'Our adjusted operating margin in the second quarter was 23.7% and we delivered another quarter

In [ ]:
vectorizer_model = CountVectorizer(stop_words="english")

In [ ]:
custom_doc = temp_dict["Rafael Tejada"]+temp_dict["Marc Casper"]+temp_dict["Stephen Williamson"]

In [ ]:
topic_model_custom = BERTopic(vectorizer_model=vectorizer_model)
topics_custom, probs_custom = topic_model_custom.fit_transform(custom_doc)

In [ ]:
topic_model_custom.get_topic_info()

,Topic,Count,Name
0,-1,1,-1_meaningfully_raising_guidance_year
1,0,164,0_business_year_growth_revenue
2,1,19,1_thanks_question_stephen_morning


In [ ]:
topic_model_custom.visualize_barchart()

In [ ]:
# topic_model_custom.visualize_topics()

# Part A: Using transformers on ThermoFisher dataset

### 1) roberta-base

In [ ]:
from flair.embeddings import TransformerDocumentEmbeddings

roberta = TransformerDocumentEmbeddings('roberta-base')
vectorizer_model = CountVectorizer(stop_words="english",token_pattern='[a-zA-Z0-9]{3,}' )
topic_model_thermofisher = BERTopic(embedding_model=roberta,vectorizer_model=vectorizer_model,top_n_words = 10,n_gram_range =(1,2))

In [ ]:
custom_data = temp_dict["Rafael Tejada"] + temp_dict["Marc Casper"] + temp_dict["Stephen Williamson"]

In [ ]:
topics_custom, probs_custom = topic_model_thermofisher.fit_transform(custom_data)

In [ ]:
topic_model_thermofisher.get_topic_info()

,Topic,Count,Name
0,-1,35,-1_business_growth_quarter_strong
1,0,53,0_right_really_think_just
2,1,30,1_strategy_progress_growth_esg
3,2,28,2_revenue_million_adjusted_year
4,3,20,3_think_appropriately_time_producing
5,4,18,4_thanks_question_thank_morning


In [ ]:
topic_model_thermofisher.visualize_topics()

In [ ]:
topic_model_thermofisher.visualize_barchart()

In [ ]:
topic_model_thermofisher.get_topic(1)

[('strategy', 0.09967831102389622),
 ('progress', 0.07589811410742474),
 ('growth', 0.07367210676744354),
 ('esg', 0.05298491433188196),
 ('proven', 0.050635311458588836),
 ('value', 0.04871880281863078),
 ('great', 0.048112317282082755),
 ('continue', 0.045240685842378295),
 ('capital', 0.044470417275431405),
 ('business', 0.04357372461313072)]

### 2) finbert

In [ ]:
#using finbert
finbert = TransformerDocumentEmbeddings('ProsusAI/finbert')
topic_model_thermofisher = BERTopic(embedding_model=finbert,vectorizer_model=vectorizer_model)

In [ ]:
topics_custom, probs_custom = topic_model_thermofisher.fit_transform(custom_data)

In [ ]:
topic_model_thermofisher.get_topic_info()

,Topic,Count,Name
0,-1,2,-1_digits_low_double_lower
1,0,166,0_year_business_quarter_growth
2,1,16,1_thanks_morning_question_thank


In [ ]:
topic_model_thermofisher.get_topic(1)

[('thanks', 0.7633609410638759),
 ('morning', 0.4069332990797141),
 ('question', 0.3871221127483987),
 ('thank', 0.3468022509065036),
 ('good', 0.30113015633109225),
 ('joining', 0.2024087697148671),
 ('matt', 0.2024087697148671),
 ('jack', 0.18956386792985594),
 ('patrick', 0.18956386792985594),
 ('puneet', 0.18045808717905987)]

In [ ]:
# topic_model_thermofisher.visualize_topics()

In [ ]:
topic_model_thermofisher.visualize_barchart()

# Part B: Using LDA  

In [ ]:
!pip install pyLDAvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 5.3 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=45eb72237a6ccf6f163cc16699f46750ebefa72c717f4fe5b8ec3d5779ce8723
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
Successfully built pyLDAvis


In [ ]:
# Sklearn
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint
# Plotting tools
import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning:

Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working



In [ ]:
import pandas as pd
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import PorterStemmer
import numpy as np
np.random.seed(2018)

In [ ]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
nltk.download("omw-1.4")

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
stemmer = PorterStemmer()

## i) LDA Analysis on Wells Fargo dataset
* It will perform lda analysis on Wells Fargo data

In [ ]:
with open("Wells_Fargo_transcript.txt","r") as f:
  string = f.read().split("\n")
  clean_string = []
  for s in string:
    if s.isspace() or s == "":
      pass
    else:
      clean_string.append(s)

cols = ["name","position"]
name_df = pd.read_csv("Wells_Fargo_names.txt",sep="\t",names=cols)

chat_list = []
temp_name = ""
temp_dict = {}
user_names = list(name_df.name)+["Operator"]
for i in clean_string:
  if i in user_names and i!="":
    temp_name = i
    if i not in temp_dict:  
      temp_dict[i]= []
  else:
    if len(i)>20:
      temp_dict[temp_name].append(i)

In [ ]:
temp_dict["Charlie Scharf"]

["Thanks very much, John. Good morning. I'll make some brief comments about our second quarter results, the operating environment, and update you on our priorities. I'll then turn the call over to Mike to review second quarter results in more detail before we take your questions.",
 'Let me start with some second quarter highlights. We earned $3.1 billion in the second quarter, our results included a $576 million impairment of equity securities, predominantly in our venture capital business due to market conditions. Revenue declined as growth in net interest income driven by rising interest rates and higher loan balances was more than offset by lower non-interest income as market conditions negatively impacted our venture capital, mortgage banking, investment banking, and wealth management advisory businesses.',
 'We continued to execute on our efficiency initiatives and our expenses declined from a year ago even with inflationary pressures and higher operating losses. We had broad-bas

In [ ]:
custom_data = temp_dict["John McDonald"] + temp_dict["Charlie Scharf"]+temp_dict["Mike Santomassimo"]

In [ ]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
data_words = list(sent_to_words(custom_data))
print(data_words[2])

['so', 'that', 'loan', 'growth', 'funding', 'is', 'coming', 'from', 'your', 'cash', 'balances', 'and', 'other', 'sources', 'of', 'liquidity', 'that', 'you', 'have']


In [ ]:
!python3 -m spacy download en_core_web_sm

2022-08-11 06:54:12.569162: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.8 MB 4.2 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])

In [ ]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ']): #'NOUN', 'ADJ', 'VERB', 'ADV'
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out

In [ ]:
data_lemmatized = lemmatization(data_words, allowed_postags=["NOUN","ADJ"]) #select noun and verb
print(data_lemmatized[:2])

['good morning guy capital next quarter basis point buffer pro forma minimum capital potential buyback capacity scb scb uncertainty level buyback', 'outlook kind cadence next quarter big step quarter third quarter fourth equation funding gap loan growth strong growth plenty cash loan growth']


In [ ]:
from nltk.stem import WordNetLemmatizer 
from nltk import word_tokenize      
class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, articles):
        return [self.wnl.lemmatize(t) for t in word_tokenize(articles)]

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
vectorizer_tfidf = TfidfVectorizer()

In [ ]:
vectorizer = CountVectorizer(analyzer='word',                                
                             min_df=10,

# minimum reqd occurences of a word 
                             stop_words='english',             
# remove stop words
                             lowercase=True,                   
# convert all words to lowercase
                             token_pattern='[a-zA-Z0-9]{3,}'  
# num chars > 3
                             # max_features=50000,             
# max number of uniq words    
)
data_vectorized = vectorizer.fit_transform(data_lemmatized)

In [ ]:
# Build LDA Model
lda_model = LatentDirichletAllocation(n_components=10,               # Number of topics
                                      max_iter=15,               
# Max learning iterations
                                      learning_method='batch',   
                                      random_state=100,          
# Random state
                                      batch_size=128,            
# n docs in each learning iter
                                      evaluate_every = -1,       
# compute perplexity every n iters, default: Don't
                                      n_jobs = -1,               
# Use all available CPUs
                                     )

In [ ]:
lda_output = lda_model.fit_transform(data_vectorized)
print(lda_output)

[[0.01250129 0.01250043 0.01250181 ... 0.01250009 0.01250008 0.01250142]
 [0.00909434 0.00909247 0.00909747 ... 0.00909106 0.00909091 0.9181641 ]
 [0.025      0.77497686 0.02500726 ... 0.02500182 0.02500512 0.02500675]
 ...
 [0.01666667 0.01666934 0.01666802 ... 0.50166411 0.01666731 0.36499538]
 [0.05       0.54996076 0.05002059 ... 0.05       0.05       0.05001375]
 [0.1        0.1        0.1        ... 0.1        0.1        0.1       ]]


In [ ]:
# Log Likelyhood: Higher the better
print("Log Likelihood: ", lda_model.score(data_vectorized))
# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
print("Perplexity: ", lda_model.perplexity(data_vectorized))
# See model parameters
pprint(lda_model.get_params())

Log Likelihood:  -5325.852544811746
Perplexity:  59.211339026479564
{'batch_size': 128,
 'doc_topic_prior': None,
 'evaluate_every': -1,
 'learning_decay': 0.7,
 'learning_method': 'batch',
 'learning_offset': 10.0,
 'max_doc_update_iter': 100,
 'max_iter': 15,
 'mean_change_tol': 0.001,
 'n_components': 10,
 'n_jobs': -1,
 'perp_tol': 0.1,
 'random_state': 100,
 'topic_word_prior': None,
 'total_samples': 1000000.0,
 'verbose': 0}


In [ ]:
# Define Search Param
search_params = {'n_components': [ 5, 10, 15, 20, 25, 30], 'learning_decay': [.4, .5, .7, .9]}
# Init the Model
lda = LatentDirichletAllocation(max_iter=5, learning_method='batch', learning_offset=50.,random_state=0)
# Init Grid Search Class
model = GridSearchCV(lda, param_grid=search_params)
# Do the Grid Search
model.fit(data_vectorized)
GridSearchCV(cv=None, error_score='raise',
       estimator=LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7, learning_method=None,
             learning_offset=10.0, max_doc_update_iter=100, max_iter=15,
             mean_change_tol=0.001, n_components=None, n_jobs=1,
      perp_tol=0.1, random_state=None,
             topic_word_prior=None, total_samples=1000000.0, verbose=0), n_jobs=1,
       param_grid={'n_components': [5,10, 15, 20, 25, 30], 'learning_decay': [0.5, 0.7, 0.9]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

GridSearchCV(error_score='raise',
             estimator=LatentDirichletAllocation(learning_method=None,
                                                 max_iter=15, n_components=None,
                                                 n_jobs=1),
             n_jobs=1,
             param_grid={'learning_decay': [0.5, 0.7, 0.9],
                         'n_components': [5, 10, 15, 20, 25, 30]},
             return_train_score='warn')

In [ ]:
# Best Model
best_lda_model = model.best_estimator_
# Model Parameters
print("Best Model's Params: ", model.best_params_)
# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)
# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(data_vectorized))

Best Model's Params:  {'learning_decay': 0.4, 'n_components': 5}
Best Log Likelihood Score:  -1367.834328602769
Model Perplexity:  57.5801131916838


In [ ]:
# Create Document — Topic Matrix
lda_output = best_lda_model.transform(data_vectorized)
# column names
topicnames = ["Topic"+ str(i) for i in range(best_lda_model.n_components)]
# index names
docnames = ["Doc" + str(i) for i in range(len(custom_data))]
# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)
# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic["dominant_topic"] = dominant_topic
# Styling
def color_green(val):
 color = "green" if val > .1 else "black"
 return "color: {col}".format(col=color)
def make_bold(val):
 weight = 700 if val > .1 else 400
 return "font-weight: {weight}".format(weight=weight)
# Apply Style
df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
df_document_topics

,Topic0,Topic1,Topic2,Topic3,Topic4,dominant_topic
Doc0,0.210000,0.030000,0.720000,0.030000,0.030000,2
Doc1,0.020000,0.020000,0.530000,0.410000,0.020000,2
Doc2,0.050000,0.050000,0.050000,0.800000,0.050000,3
Doc3,0.030000,0.040000,0.860000,0.030000,0.030000,2
Doc4,0.010000,0.010000,0.200000,0.110000,0.670000,4
Doc5,0.010000,0.010000,0.010000,0.950000,0.010000,3
Doc6,0.010000,0.260000,0.010000,0.710000,0.010000,3
Doc7,0.040000,0.240000,0.040000,0.630000,0.040000,3
Doc8,0.760000,0.200000,0.010000,0.010000,0.010000,0
Doc9,0.190000,0.020000,0.020000,0.020000,0.750000,4


In [ ]:
# Topic-Keyword Matrix
df_topic_keywords = pd.DataFrame(best_lda_model.components_)
# Assign Column and Index
df_topic_keywords.columns = vectorizer.get_feature_names()
df_topic_keywords.index = topicnames
# View
df_topic_keywords.head()

,asset,average,balance,banking,basis,beta,big,bit,business,capital,...,risk,second,slide,small,strong,term,thing,time,way,year
Topic0,0.652627,0.200306,5.623077,0.200344,0.200465,6.182974,7.365110,0.228474,1.397737,1.768933,...,2.817427,0.203232,0.215821,3.653014,2.338822,4.760652,0.201499,9.693730,0.201393,14.325046
Topic1,0.200011,1.232228,0.203967,0.200637,0.200008,6.209050,3.221313,0.200008,31.065233,0.200019,...,3.773405,0.202989,0.201805,7.603465,0.204068,6.305427,12.799162,3.686509,10.017728,25.306015
Topic2,7.598298,0.238693,2.494840,3.982655,11.514854,0.200015,4.011943,14.170637,0.717636,7.532247,...,0.200012,18.393235,0.200610,2.898055,0.764977,5.465203,6.599186,0.219069,1.373932,10.167463
Topic3,8.347292,15.127773,14.963389,11.956917,0.626788,0.201739,0.201618,0.200231,0.206221,4.021020,...,4.987746,0.202845,2.921122,2.644457,4.016973,0.200014,0.200137,0.200014,0.200295,21.748575
Topic4,0.201773,0.201000,1.714727,7.659447,2.457884,0.206223,0.200017,0.200649,13.613173,10.477781,...,0.221410,9.997700,8.460642,0.201010,11.675160,1.268704,0.200015,0.200679,0.206652,19.452901


In [ ]:
# Show top n keywords for each topic
def show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=5):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords
topic_keywords = show_topics(vectorizer=vectorizer, lda_model=best_lda_model, n_words=5)
# Topic - Keywords Dataframe
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords

,Word 0,Word 1,Word 2,Word 3,Word 4
Topic 0,card,credit,year,new,deposit
Topic 1,business,year,customer,product,expense
Topic 2,quarter,point,second,rate,little
Topic 3,loan,year,high,average,balance
Topic 4,rate,income,net,year,commercial


In [ ]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(best_lda_model, data_vectorized, vectorizer)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.075905  0.041708       1        1  24.797803
2     -0.150106 -0.072364       2        1  23.741290
4     -0.141372  0.040281       3        1  22.140927
1      0.182887  0.140521       4        1  16.529235
0      0.184496 -0.150146       5        1  12.790745, topic_info=        Term       Freq      Total Category  logprob  loglift
42   quarter  78.000000  78.000000  Default  30.0000  30.0000
10      card  24.000000  24.000000  Default  29.0000  29.0000
8   business  43.000000  43.000000  Default  28.0000  28.0000
15  customer  26.000000  26.000000  Default  27.0000  27.0000
24    income  25.000000  25.000000  Default  26.0000  26.0000
..       ...        ...        ...      ...      ...      ...
34    market   2.039000  33.954886   Topic5  -4.4051  -0.7561
33       low   1.198058  28.014360   Topic5  -4.9368  -1.0956
21    growth   1.207017  29.713069   Topic5  -4.9294  -1.1470
0      asset   0.609488  16.690519   Topic5  -5.6126  -1.2535
8   business   1.305346  43.451591   Topic5  -4.8510  -1.4487

[185 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
0         1  0.539228    asset
0         2  0.419400    asset
0         5  0.059914    asset
1         1  0.907281  average
1         4  0.056705  average
...     ...       ...      ...
55        1  0.263279     year
55        2  0.103022     year
55        3  0.217491     year
55        4  0.263279     year
55        5  0.148810     year

[172 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 3, 5, 2, 1])

## ii) LDA Analysis on Cisco dataset
* It will perform lda analysis on cisco data

In [ ]:
with open("Cisco_transcript.txt","r") as f:
  string = f.read().split("\n")
  clean_string = []
  for s in string:
    if s.isspace() or s == "":
      pass
    else:
      clean_string.append(s)

cols = ["name","position"]
name_df = pd.read_csv("Cisco_names.txt",sep="\t",names=cols)

chat_list = []
temp_name = ""
temp_dict = {}
user_names = list(name_df.name)+["Operator"]
for i in clean_string:
  if i in user_names and i!="":
    temp_name = i
    if i not in temp_dict:  
      temp_dict[i]= []
  else:
    if len(i)>20:
      temp_dict[temp_name].append(i)

###observation 
* dividing the dialogs on the basis of its original form is better since combining all the dialogs seems to create worst results


In [ ]:
temp_dict["Chuck Robbins"]

['Thanks, Marilyn, and good afternoon, everyone. When we spoke with you back in February, we entered Q3 in the second half of our fiscal year with optimism despite the supply and component challenges and other headwinds impacting us and many of our peers. Many of those factors that fueled that optimism remain unchanged today.',
 'We continue to see strong demand resulting in record backlog, our business transformation is progressing well and our differentiated innovation across our portfolio is helping our customers embrace and adopt the multiple technology transitions happening.',
 'However, there were two unanticipated events since our last earnings call, which impacted our Q3 revenue performance.',
 'The first is the war in Ukraine. This resulted in us ceasing operations in Russia and Belarus and had a corresponding revenue impact, which Scott will discuss.',
 'The second relates to COVID related lockdowns in China, which began in late March. These lockdowns resulted in an even more

In [ ]:
custom_data = temp_dict["Chuck Robbins"] + temp_dict["Marilyn Mora"]+temp_dict["Scott Herren"]

In [ ]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
data_words = list(sent_to_words(custom_data))
print(data_words[2])

['however', 'there', 'were', 'two', 'unanticipated', 'events', 'since', 'our', 'last', 'earnings', 'call', 'which', 'impacted', 'our', 'revenue', 'performance']


In [ ]:
import spacy

In [ ]:
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])

In [ ]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ']): #'NOUN', 'ADJ', 'VERB', 'ADV'
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out

In [ ]:
data_lemmatized = lemmatization(data_words, allowed_postags=["NOUN","ADJ"]) #select noun and verb
print(data_lemmatized[:2])

['thank good afternoon second half fiscal year optimism supply component challenge other headwind many peer many factor optimism unchanged today', 'strong demand record backlog business transformation differentiated innovation portfolio customer multiple technology transition']


In [ ]:
from nltk.stem import WordNetLemmatizer 
from nltk import word_tokenize      
class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, articles):
        return [self.wnl.lemmatize(t) for t in word_tokenize(articles)]

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
vectorizer_tfidf = TfidfVectorizer()

In [ ]:
vectorizer = CountVectorizer(analyzer='word',min_df=10,stop_words='english', lowercase=True,token_pattern='[a-zA-Z0-9]{3,}'  )
data_vectorized = vectorizer.fit_transform(data_lemmatized)
print(data_vectorized)

  (0, 8)	1
  (0, 31)	1
  (0, 24)	1
  (0, 4)	1
  (1, 23)	1
  (1, 6)	1
  (1, 0)	1
  (1, 2)	1
  (1, 16)	1
  (1, 5)	1
  (2, 20)	1
  (3, 20)	1
  (3, 11)	1
  (4, 4)	1
  (4, 5)	1
  (4, 17)	1
  (5, 20)	1
  (5, 11)	1
  (5, 10)	1
  (6, 24)	1
  (6, 6)	1
  (6, 20)	1
  (6, 26)	1
  (6, 18)	2
  (6, 28)	1
  :	:
  (175, 13)	1
  (176, 27)	1
  (177, 6)	1
  (179, 0)	1
  (180, 8)	1
  (180, 24)	2
  (180, 4)	1
  (180, 5)	1
  (180, 17)	2
  (180, 26)	1
  (180, 25)	1
  (180, 19)	1
  (180, 3)	1
  (181, 4)	1
  (182, 0)	1
  (183, 20)	1
  (183, 11)	1
  (183, 18)	1
  (183, 19)	1
  (184, 24)	1
  (184, 18)	1
  (184, 19)	1
  (184, 3)	1
  (185, 26)	1
  (186, 19)	1


In [ ]:
# Build LDA Model
lda_model = LatentDirichletAllocation(n_components=10,               # Number of topics
                                      max_iter=15,               
# Max learning iterations
                                      learning_method='batch',   
                                      random_state=100,          
# Random state
                                      batch_size=128,            
# n docs in each learning iter
                                      evaluate_every = -1,       
# compute perplexity every n iters, default: Don't
                                      n_jobs = -1,               
# Use all available CPUs
                                     )
lda_output = lda_model.fit_transform(data_vectorized)
print(lda_output)

[[0.02000104 0.02       0.02000083 ... 0.81997055 0.02000526 0.02001359]
 [0.01428732 0.01428648 0.01428625 ... 0.01428571 0.01428585 0.57877954]
 [0.05       0.05       0.05001399 ... 0.05       0.05       0.05      ]
 ...
 [0.02       0.02000673 0.22000003 ... 0.02000073 0.61998427 0.02000061]
 [0.05       0.05       0.05       ... 0.05       0.05000828 0.54998672]
 [0.05       0.05       0.55       ... 0.05       0.05       0.05      ]]


In [ ]:
lda_output

array([[0.02000104, 0.02      , 0.02000083, ..., 0.81997055, 0.02000526,
        0.02001359],
       [0.01428732, 0.01428648, 0.01428625, ..., 0.01428571, 0.01428585,
        0.57877954],
       [0.05      , 0.05      , 0.05001399, ..., 0.05      , 0.05      ,
        0.05      ],
       ...,
       [0.02      , 0.02000673, 0.22000003, ..., 0.02000073, 0.61998427,
        0.02000061],
       [0.05      , 0.05      , 0.05      , ..., 0.05      , 0.05000828,
        0.54998672],
       [0.05      , 0.05      , 0.55      , ..., 0.05      , 0.05      ,
        0.05      ]])

In [ ]:
# Log Likelyhood: Higher the better
print("Log Likelihood: ", lda_model.score(data_vectorized))
# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
print("Perplexity: ", lda_model.perplexity(data_vectorized))
# See model parameters
pprint(lda_model.get_params())

Log Likelihood:  -2463.9717030867414
Perplexity:  38.28003783127336
{'batch_size': 128,
 'doc_topic_prior': None,
 'evaluate_every': -1,
 'learning_decay': 0.7,
 'learning_method': 'batch',
 'learning_offset': 10.0,
 'max_doc_update_iter': 100,
 'max_iter': 15,
 'mean_change_tol': 0.001,
 'n_components': 10,
 'n_jobs': -1,
 'perp_tol': 0.1,
 'random_state': 100,
 'topic_word_prior': None,
 'total_samples': 1000000.0,
 'verbose': 0}


In [ ]:
# Define Search Param
search_params = {'n_components': [ 5, 10, 15, 20, 25, 30], 'learning_decay': [.4, .5, .7, .9]}
# Init the Model
lda = LatentDirichletAllocation(max_iter=5, learning_method='batch', learning_offset=50.,random_state=0)
# Init Grid Search Class
model = GridSearchCV(lda, param_grid=search_params)
# Do the Grid Search
model.fit(data_vectorized)
GridSearchCV(cv=None, error_score='raise',
       estimator=LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7, learning_method=None,
             learning_offset=10.0, max_doc_update_iter=100, max_iter=15,
             mean_change_tol=0.001, n_components=None, n_jobs=1,
      perp_tol=0.1, random_state=None,
             topic_word_prior=None, total_samples=1000000.0, verbose=0), n_jobs=1,
       param_grid={'n_components': [5,10, 15, 20, 25, 30], 'learning_decay': [0.5, 0.7, 0.9]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

GridSearchCV(error_score='raise',
             estimator=LatentDirichletAllocation(learning_method=None,
                                                 max_iter=15, n_components=None,
                                                 n_jobs=1),
             n_jobs=1,
             param_grid={'learning_decay': [0.5, 0.7, 0.9],
                         'n_components': [5, 10, 15, 20, 25, 30]},
             return_train_score='warn')

In [ ]:
# Best Model
best_lda_model = model.best_estimator_
# Model Parameters
print("Best Model's Params: ", model.best_params_)
# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)
# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(data_vectorized))

Best Model's Params:  {'learning_decay': 0.4, 'n_components': 5}
Best Log Likelihood Score:  -638.4655157377955
Model Perplexity:  37.60799500301649


In [ ]:
# Create Document — Topic Matrix
lda_output = best_lda_model.transform(data_vectorized)
# column names
topicnames = ["Topic"+ str(i) for i in range(best_lda_model.n_components)]
# index names
docnames = ["Doc" + str(i) for i in range(len(custom_data))]
# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)
# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic["dominant_topic"] = dominant_topic
# Styling
def color_green(val):
 color = "green" if val > .1 else "black"
 return "color: {col}".format(col=color)
def make_bold(val):
 weight = 700 if val > .1 else 400
 return "font-weight: {weight}".format(weight=weight)
# Apply Style
df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
df_document_topics

,Topic0,Topic1,Topic2,Topic3,Topic4,dominant_topic
Doc0,0.040000,0.040000,0.040000,0.400000,0.480000,4
Doc1,0.340000,0.570000,0.030000,0.030000,0.030000,1
Doc2,0.600000,0.100000,0.100000,0.100000,0.100000,0
Doc3,0.730000,0.070000,0.070000,0.070000,0.070000,0
Doc4,0.050000,0.050000,0.800000,0.050000,0.050000,2
Doc5,0.530000,0.050000,0.050000,0.050000,0.310000,0
Doc6,0.120000,0.130000,0.030000,0.550000,0.170000,3
Doc7,0.040000,0.040000,0.210000,0.040000,0.670000,4
Doc8,0.100000,0.100000,0.100000,0.600000,0.100000,3
Doc9,0.370000,0.180000,0.080000,0.020000,0.360000,0


In [ ]:
# Topic-Keyword Matrix
df_topic_keywords = pd.DataFrame(best_lda_model.components_)
# Assign Column and Index
df_topic_keywords.columns = vectorizer.get_feature_names()
df_topic_keywords.index = topicnames
# View
df_topic_keywords.head()

,backlog,basis,business,chain,component,customer,demand,enterprise,good,growth,...,solid,strong,supply,team,term,thing,time,way,week,year
Topic0,17.953126,4.370770,4.532611,1.229756,0.200245,0.201941,2.394677,0.202740,0.200625,26.040001,...,6.819866,2.231446,0.201048,0.200024,1.254635,0.203938,0.200976,0.201943,8.296933,13.775827
Topic1,0.202176,0.202105,9.364425,0.200484,0.200112,4.771128,18.712452,0.345982,5.558917,0.898184,...,3.577977,8.105083,0.200596,0.201111,0.200390,0.271031,0.286196,7.199526,0.200025,2.530601
Topic2,0.200023,10.023756,4.205231,0.202442,9.574689,24.613842,0.200980,15.049877,0.212499,0.213123,...,0.200811,1.255650,1.404049,5.684033,2.789821,9.782246,0.200706,0.202157,2.451945,1.480756
Topic3,0.201612,0.200904,0.200399,10.166998,8.202029,0.202287,1.248421,0.201308,0.201426,4.813188,...,0.200484,1.205411,28.208632,2.836925,0.204673,4.230462,19.106194,5.166622,3.848362,3.305443
Topic4,0.443063,0.202464,2.697334,0.200320,1.822925,0.210803,0.443470,0.200094,5.826533,5.035504,...,0.200861,0.202410,0.985676,5.077908,12.550481,5.512322,1.205928,0.229751,0.202735,33.907374


In [ ]:
# Show top n keywords for each topic
def show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=5):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords
topic_keywords = show_topics(vectorizer=vectorizer, lda_model=best_lda_model, n_words=5)
# Topic - Keywords Dataframe
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords

,Word 0,Word 1,Word 2,Word 3,Word 4
Topic 0,revenue,growth,product,backlog,software
Topic 1,question,demand,business,strong,way
Topic 2,customer,enterprise,product,basis,thing
Topic 3,quarter,supply,time,chain,component
Topic 4,year,guidance,term,quarter,good


In [ ]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(best_lda_model, data_vectorized, vectorizer)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.201933 -0.090046       1        1  30.480209
3     -0.060921  0.227056       2        1  20.155355
2      0.102239 -0.073549       3        1  18.007547
4      0.006116  0.092806       4        1  17.886652
1     -0.249368 -0.156266       5        1  13.470237, topic_info=        Term       Freq      Total Category  logprob  loglift
31      year  54.000000  54.000000  Default  30.0000  30.0000
19  question  21.000000  21.000000  Default  29.0000  29.0000
5   customer  28.000000  28.000000  Default  28.0000  28.0000
20   revenue  39.000000  39.000000  Default  27.0000  27.0000
6     demand  21.000000  21.000000  Default  26.0000  26.0000
..       ...        ...        ...      ...      ...      ...
15     point   0.192815  21.366744   Topic5  -6.1575  -2.7032
11    impact   0.191877  23.075092   Topic5  -6.1624  -2.7850
24    supply   0.191686  27.715794   Topic5  -6.1634  -2.9692
20   revenue   0.191594  39.648512   Topic5  -6.1639  -3.3277
17   product   0.191589  33.635468   Topic5  -6.1639  -3.1633

[190 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
0         1  0.921570   backlog
1         1  0.279727     basis
1         3  0.629385     basis
2         1  0.197356  business
2         3  0.197356  business
...     ...       ...       ...
31        1  0.236879      year
31        2  0.054664      year
31        3  0.018221      year
31        4  0.637752      year
31        5  0.036443      year

[87 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 4, 3, 5, 2])

## iii) LDA Analysis on Intuit dataset
* It will perform lda analysis on Intuit data

In [ ]:
with open("Intuit_transcript.txt","r") as f:
  string = f.read().split("\n")
  clean_string = []
  for s in string:
    if s.isspace() or s == "":
      pass
    else:
      clean_string.append(s)

cols = ["name","position"]
name_df = pd.read_csv("Intuit_names.txt",sep="\t",names=cols)

chat_list = []
temp_name = ""
temp_dict = {}
user_names = list(name_df.name)+["Operator"]
for i in clean_string:
  if i in user_names and i!="":
    temp_name = i
    if i not in temp_dict:  
      temp_dict[i]= []
  else:
    if len(i)>20:
      temp_dict[temp_name].append(i)

In [ ]:
temp_dict["Sasan Goodarzi"]

['Thanks, Kim and thanks to all of you for joining us today. I am proud of our continued momentum as we execute on our strategy to be the global AI-driven expert platform powering the prosperity of consumers and small businesses.',
 'We have a nearly $300 billion addressable market driven by tailwinds that include shifts to virtual solutions, acceleration to online and omnichannel capabilities and digital money offerings. This, combined with the team’s execution, is contributing to the strength of our performance.',
 'Third quarter revenue grew 35%, including 6 points from the addition of Mailchimp. It was another strong quarter for our Small Business and Self-Employed Group with revenue up 42%, 20% organically. Credit Karma posted another quarter with revenue at an all-time high, up 48%. And I am proud of how the team executed in the Consumer Group in another unusual tax season.',
 'We are confident in our business trajectory and are raising our total Intuit revenue and non-GAAP earni

In [ ]:
custom_doc = temp_dict["Sasan Goodarzi"]+temp_dict["Kim Watkins"]+temp_dict["Michelle Clatterbuck"]

In [ ]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
data_words = list(sent_to_words(custom_data))
print(data_words[2])

['however', 'there', 'were', 'two', 'unanticipated', 'events', 'since', 'our', 'last', 'earnings', 'call', 'which', 'impacted', 'our', 'revenue', 'performance']


In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ']): #'NOUN', 'ADJ', 'VERB', 'ADV'
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out

In [ ]:
data_lemmatized = lemmatization(data_words, allowed_postags=["NOUN","ADJ"]) #select noun and verb
print(data_lemmatized[:2])

['thank good afternoon second half fiscal year optimism supply component challenge other headwind many peer many factor optimism unchanged today', 'strong demand record backlog business transformation differentiated innovation portfolio customer multiple technology transition']


In [ ]:
from nltk.stem import WordNetLemmatizer 
from nltk import word_tokenize      
class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, articles):
        return [self.wnl.lemmatize(t) for t in word_tokenize(articles)]

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
vectorizer_tfidf = TfidfVectorizer()
vectorizer = CountVectorizer(analyzer='word',min_df=10,
                             stop_words='english',             
                             lowercase=True,                   
                             token_pattern='[a-zA-Z0-9]{3,}')
data_vectorized = vectorizer.fit_transform(data_lemmatized)
print(data_vectorized)

  (0, 8)	1
  (0, 31)	1
  (0, 24)	1
  (0, 4)	1
  (1, 23)	1
  (1, 6)	1
  (1, 0)	1
  (1, 2)	1
  (1, 16)	1
  (1, 5)	1
  (2, 20)	1
  (3, 20)	1
  (3, 11)	1
  (4, 4)	1
  (4, 5)	1
  (4, 17)	1
  (5, 20)	1
  (5, 11)	1
  (5, 10)	1
  (6, 24)	1
  (6, 6)	1
  (6, 20)	1
  (6, 26)	1
  (6, 18)	2
  (6, 28)	1
  :	:
  (175, 13)	1
  (176, 27)	1
  (177, 6)	1
  (179, 0)	1
  (180, 8)	1
  (180, 24)	2
  (180, 4)	1
  (180, 5)	1
  (180, 17)	2
  (180, 26)	1
  (180, 25)	1
  (180, 19)	1
  (180, 3)	1
  (181, 4)	1
  (182, 0)	1
  (183, 20)	1
  (183, 11)	1
  (183, 18)	1
  (183, 19)	1
  (184, 24)	1
  (184, 18)	1
  (184, 19)	1
  (184, 3)	1
  (185, 26)	1
  (186, 19)	1


In [ ]:
# Build LDA Model
lda_model = LatentDirichletAllocation(n_components=10,               # Number of topics
                                      max_iter=15,               
                                      learning_method='batch',   
                                      random_state=100,          
                                      batch_size=128,            
                                      evaluate_every = -1,       
                                      n_jobs = -1,               
                                     )
lda_output = lda_model.fit_transform(data_vectorized)
print(lda_output)

[[0.02000104 0.02       0.02000083 ... 0.81997055 0.02000526 0.02001359]
 [0.01428732 0.01428648 0.01428625 ... 0.01428571 0.01428585 0.57877954]
 [0.05       0.05       0.05001399 ... 0.05       0.05       0.05      ]
 ...
 [0.02       0.02000673 0.22000003 ... 0.02000073 0.61998427 0.02000061]
 [0.05       0.05       0.05       ... 0.05       0.05000828 0.54998672]
 [0.05       0.05       0.55       ... 0.05       0.05       0.05      ]]


In [ ]:
# Log Likelyhood: Higher the better
print("Log Likelihood: ", lda_model.score(data_vectorized))
# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
print("Perplexity: ", lda_model.perplexity(data_vectorized))
# See model parameters
pprint(lda_model.get_params())

Log Likelihood:  -2463.9717030867414
Perplexity:  38.28003783127336
{'batch_size': 128,
 'doc_topic_prior': None,
 'evaluate_every': -1,
 'learning_decay': 0.7,
 'learning_method': 'batch',
 'learning_offset': 10.0,
 'max_doc_update_iter': 100,
 'max_iter': 15,
 'mean_change_tol': 0.001,
 'n_components': 10,
 'n_jobs': -1,
 'perp_tol': 0.1,
 'random_state': 100,
 'topic_word_prior': None,
 'total_samples': 1000000.0,
 'verbose': 0}


In [ ]:
# Define Search Param
search_params = {'n_components': [ 5, 10, 15, 20, 25, 30], 'learning_decay': [.4, .5, .7, .9]}
# Init the Model
lda = LatentDirichletAllocation(max_iter=5, learning_method='batch', learning_offset=50.,random_state=0)
# Init Grid Search Class
model = GridSearchCV(lda, param_grid=search_params)
# Do the Grid Search
model.fit(data_vectorized)
GridSearchCV(cv=None, error_score='raise',
       estimator=LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7, learning_method=None,
             learning_offset=10.0, max_doc_update_iter=100, max_iter=15,
             mean_change_tol=0.001, n_components=None, n_jobs=1,
      perp_tol=0.1, random_state=None,
             topic_word_prior=None, total_samples=1000000.0, verbose=0), n_jobs=1,
       param_grid={'n_components': [5,10, 15, 20, 25, 30], 'learning_decay': [0.5, 0.7, 0.9]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

GridSearchCV(error_score='raise',
             estimator=LatentDirichletAllocation(learning_method=None,
                                                 max_iter=15, n_components=None,
                                                 n_jobs=1),
             n_jobs=1,
             param_grid={'learning_decay': [0.5, 0.7, 0.9],
                         'n_components': [5, 10, 15, 20, 25, 30]},
             return_train_score='warn')

In [ ]:
# Best Model
best_lda_model = model.best_estimator_
# Model Parameters
print("Best Model's Params: ", model.best_params_)
# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)
# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(data_vectorized))

Best Model's Params:  {'learning_decay': 0.4, 'n_components': 5}
Best Log Likelihood Score:  -638.4655157377955
Model Perplexity:  37.60799500301649


In [ ]:
# Create Document — Topic Matrix
lda_output = best_lda_model.transform(data_vectorized)
# column names
topicnames = ["Topic"+ str(i) for i in range(best_lda_model.n_components)]
# index names
docnames = ["Doc" + str(i) for i in range(len(custom_data))]
# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)
# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic["dominant_topic"] = dominant_topic
# Styling
def color_green(val):
 color = "green" if val > .1 else "black"
 return "color: {col}".format(col=color)
def make_bold(val):
 weight = 700 if val > .1 else 400
 return "font-weight: {weight}".format(weight=weight)
# Apply Style
df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
df_document_topics

,Topic0,Topic1,Topic2,Topic3,Topic4,dominant_topic
Doc0,0.040000,0.040000,0.040000,0.400000,0.480000,4
Doc1,0.340000,0.570000,0.030000,0.030000,0.030000,1
Doc2,0.600000,0.100000,0.100000,0.100000,0.100000,0
Doc3,0.730000,0.070000,0.070000,0.070000,0.070000,0
Doc4,0.050000,0.050000,0.800000,0.050000,0.050000,2
Doc5,0.530000,0.050000,0.050000,0.050000,0.310000,0
Doc6,0.120000,0.130000,0.030000,0.550000,0.170000,3
Doc7,0.040000,0.040000,0.210000,0.040000,0.670000,4
Doc8,0.100000,0.100000,0.100000,0.600000,0.100000,3
Doc9,0.370000,0.180000,0.080000,0.020000,0.360000,0


In [ ]:
# Topic-Keyword Matrix
df_topic_keywords = pd.DataFrame(best_lda_model.components_)
# Assign Column and Index
df_topic_keywords.columns = vectorizer.get_feature_names()
df_topic_keywords.index = topicnames
# View
df_topic_keywords.head()

,backlog,basis,business,chain,component,customer,demand,enterprise,good,growth,...,solid,strong,supply,team,term,thing,time,way,week,year
Topic0,17.953126,4.370770,4.532611,1.229756,0.200245,0.201941,2.394677,0.202740,0.200625,26.040001,...,6.819866,2.231446,0.201048,0.200024,1.254635,0.203938,0.200976,0.201943,8.296933,13.775827
Topic1,0.202176,0.202105,9.364425,0.200484,0.200112,4.771128,18.712452,0.345982,5.558917,0.898184,...,3.577977,8.105083,0.200596,0.201111,0.200390,0.271031,0.286196,7.199526,0.200025,2.530601
Topic2,0.200023,10.023756,4.205231,0.202442,9.574689,24.613842,0.200980,15.049877,0.212499,0.213123,...,0.200811,1.255650,1.404049,5.684033,2.789821,9.782246,0.200706,0.202157,2.451945,1.480756
Topic3,0.201612,0.200904,0.200399,10.166998,8.202029,0.202287,1.248421,0.201308,0.201426,4.813188,...,0.200484,1.205411,28.208632,2.836925,0.204673,4.230462,19.106194,5.166622,3.848362,3.305443
Topic4,0.443063,0.202464,2.697334,0.200320,1.822925,0.210803,0.443470,0.200094,5.826533,5.035504,...,0.200861,0.202410,0.985676,5.077908,12.550481,5.512322,1.205928,0.229751,0.202735,33.907374


In [ ]:
# Show top n keywords for each topic
def show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=5):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords
topic_keywords = show_topics(vectorizer=vectorizer, lda_model=best_lda_model, n_words=5)
# Topic - Keywords Dataframe
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords

,Word 0,Word 1,Word 2,Word 3,Word 4
Topic 0,revenue,growth,product,backlog,software
Topic 1,question,demand,business,strong,way
Topic 2,customer,enterprise,product,basis,thing
Topic 3,quarter,supply,time,chain,component
Topic 4,year,guidance,term,quarter,good


In [ ]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(best_lda_model, data_vectorized, vectorizer)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.201933 -0.090046       1        1  30.480209
3     -0.060921  0.227056       2        1  20.155355
2      0.102239 -0.073549       3        1  18.007547
4      0.006116  0.092806       4        1  17.886652
1     -0.249368 -0.156266       5        1  13.470237, topic_info=        Term       Freq      Total Category  logprob  loglift
31      year  54.000000  54.000000  Default  30.0000  30.0000
19  question  21.000000  21.000000  Default  29.0000  29.0000
5   customer  28.000000  28.000000  Default  28.0000  28.0000
20   revenue  39.000000  39.000000  Default  27.0000  27.0000
6     demand  21.000000  21.000000  Default  26.0000  26.0000
..       ...        ...        ...      ...      ...      ...
15     point   0.192815  21.366744   Topic5  -6.1575  -2.7032
11    impact   0.191877  23.075092   Topic5  -6.1624  -2.7850
24    supply   0.191686  27.715794   Topic5  -6.1634  -2.9692
20   revenue   0.191594  39.648512   Topic5  -6.1639  -3.3277
17   product   0.191589  33.635468   Topic5  -6.1639  -3.1633

[190 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
0         1  0.921570   backlog
1         1  0.279727     basis
1         3  0.629385     basis
2         1  0.197356  business
2         3  0.197356  business
...     ...       ...       ...
31        1  0.236879      year
31        2  0.054664      year
31        3  0.018221      year
31        4  0.637752      year
31        5  0.036443      year

[87 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 4, 3, 5, 2])

## iv) LDA Analysis on UHG dataset

In [ ]:
with open("UHG_transcripts.txt","r") as f:
  string = f.read().split("\n")
  clean_string = []
  for s in string:
    if s.isspace() or s == "":
      pass
    else:
      clean_string.append(s)

cols = ["name","position"]
name_df = pd.read_csv("UHG_names.txt",sep="\t",names=cols)

chat_list = []
temp_name = ""
temp_dict = {}
user_names = list(name_df.name)+["Operator"]
for i in clean_string:
  if i in user_names and i!="":
    temp_name = i
    if i not in temp_dict:  
      temp_dict[i]= []
  else:
    if len(i)>20:
      temp_dict[temp_name].append(i)

In [ ]:
temp_dict["Andrew Witty"]

['Katy, thank you, and good morning, everybody, and thank you for joining us today. UnitedHealth Group ended the second half of the year with sustained momentum as we execute on our objective to serve more people more effectively with connected high-quality care.',
 "For that, I want to thank our 360,000 colleagues. It's their unwavering commitment to our mission and their hard work and support of the people we serve that makes all of this possible.",
 "As a result of the strong performance at both Optum and UnitedHealthcare, we are increasing our adjusted earnings per share outlook for the year to a range of $21.40 to $21.90 per share. Comprehensive value-based care is a central theme of our growth strategy, helping more patients and care providers transition from traditional fee-for-service to a value-based orientation. We aim to drive better and more consistent care outcomes at lower overall cost, often for people who are among society's most vulnerable with multiple chronic conditi

In [ ]:
custom_doc = temp_dict["Andrew Witty"]+temp_dict["Dirk McMahon"]+temp_dict["John Rex"]

In [ ]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
data_words = list(sent_to_words(custom_data))
print(data_words[2])

['however', 'there', 'were', 'two', 'unanticipated', 'events', 'since', 'our', 'last', 'earnings', 'call', 'which', 'impacted', 'our', 'revenue', 'performance']


In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ']): #'NOUN', 'ADJ', 'VERB', 'ADV'
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out

In [ ]:
data_lemmatized = lemmatization(data_words, allowed_postags=["NOUN","ADJ"]) #select noun and verb
print(data_lemmatized[:2])

['thank good afternoon second half fiscal year optimism supply component challenge other headwind many peer many factor optimism unchanged today', 'strong demand record backlog business transformation differentiated innovation portfolio customer multiple technology transition']


In [ ]:
from nltk.stem import WordNetLemmatizer 
from nltk import word_tokenize      
class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, articles):
        return [self.wnl.lemmatize(t) for t in word_tokenize(articles)]

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
vectorizer_tfidf = TfidfVectorizer()

In [ ]:
vectorizer = CountVectorizer(analyzer='word',min_df=10,
                             stop_words='english',             
                             lowercase=True,                   
                             token_pattern='[a-zA-Z0-9]{3,}')
data_vectorized = vectorizer.fit_transform(data_lemmatized)
print(data_vectorized)

  (0, 8)	1
  (0, 31)	1
  (0, 24)	1
  (0, 4)	1
  (1, 23)	1
  (1, 6)	1
  (1, 0)	1
  (1, 2)	1
  (1, 16)	1
  (1, 5)	1
  (2, 20)	1
  (3, 20)	1
  (3, 11)	1
  (4, 4)	1
  (4, 5)	1
  (4, 17)	1
  (5, 20)	1
  (5, 11)	1
  (5, 10)	1
  (6, 24)	1
  (6, 6)	1
  (6, 20)	1
  (6, 26)	1
  (6, 18)	2
  (6, 28)	1
  :	:
  (175, 13)	1
  (176, 27)	1
  (177, 6)	1
  (179, 0)	1
  (180, 8)	1
  (180, 24)	2
  (180, 4)	1
  (180, 5)	1
  (180, 17)	2
  (180, 26)	1
  (180, 25)	1
  (180, 19)	1
  (180, 3)	1
  (181, 4)	1
  (182, 0)	1
  (183, 20)	1
  (183, 11)	1
  (183, 18)	1
  (183, 19)	1
  (184, 24)	1
  (184, 18)	1
  (184, 19)	1
  (184, 3)	1
  (185, 26)	1
  (186, 19)	1


In [ ]:
# Build LDA Model
lda_model = LatentDirichletAllocation(n_components=10,               # Number of topics
                                      max_iter=15,               
# Max learning iterations
                                      learning_method='batch',   
                                      random_state=100,          
# Random state
                                      batch_size=128,            
# n docs in each learning iter
                                      evaluate_every = -1,       
# compute perplexity every n iters, default: Don't
                                      n_jobs = -1,               
# Use all available CPUs
                                     )
lda_output = lda_model.fit_transform(data_vectorized)
print(lda_output)

[[0.02000104 0.02       0.02000083 ... 0.81997055 0.02000526 0.02001359]
 [0.01428732 0.01428648 0.01428625 ... 0.01428571 0.01428585 0.57877954]
 [0.05       0.05       0.05001399 ... 0.05       0.05       0.05      ]
 ...
 [0.02       0.02000673 0.22000003 ... 0.02000073 0.61998427 0.02000061]
 [0.05       0.05       0.05       ... 0.05       0.05000828 0.54998672]
 [0.05       0.05       0.55       ... 0.05       0.05       0.05      ]]


In [ ]:
# Log Likelyhood: Higher the better
print("Log Likelihood: ", lda_model.score(data_vectorized))
# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
print("Perplexity: ", lda_model.perplexity(data_vectorized))
# See model parameters
pprint(lda_model.get_params())

Log Likelihood:  -2463.9717030867414
Perplexity:  38.28003783127336
{'batch_size': 128,
 'doc_topic_prior': None,
 'evaluate_every': -1,
 'learning_decay': 0.7,
 'learning_method': 'batch',
 'learning_offset': 10.0,
 'max_doc_update_iter': 100,
 'max_iter': 15,
 'mean_change_tol': 0.001,
 'n_components': 10,
 'n_jobs': -1,
 'perp_tol': 0.1,
 'random_state': 100,
 'topic_word_prior': None,
 'total_samples': 1000000.0,
 'verbose': 0}


In [ ]:
# Define Search Param
search_params = {'n_components': [ 5, 10, 15, 20, 25, 30], 'learning_decay': [.4, .5, .7, .9]}
# Init the Model
lda = LatentDirichletAllocation(max_iter=5, learning_method='batch', learning_offset=50.,random_state=0)
# Init Grid Search Class
model = GridSearchCV(lda, param_grid=search_params)
# Do the Grid Search
model.fit(data_vectorized)
GridSearchCV(cv=None, error_score='raise',
       estimator=LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7, learning_method=None,
             learning_offset=10.0, max_doc_update_iter=100, max_iter=15,
             mean_change_tol=0.001, n_components=None, n_jobs=1,
      perp_tol=0.1, random_state=None,
             topic_word_prior=None, total_samples=1000000.0, verbose=0), n_jobs=1,
       param_grid={'n_components': [5,10, 15, 20, 25, 30], 'learning_decay': [0.5, 0.7, 0.9]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

GridSearchCV(error_score='raise',
             estimator=LatentDirichletAllocation(learning_method=None,
                                                 max_iter=15, n_components=None,
                                                 n_jobs=1),
             n_jobs=1,
             param_grid={'learning_decay': [0.5, 0.7, 0.9],
                         'n_components': [5, 10, 15, 20, 25, 30]},
             return_train_score='warn')

In [ ]:
# Best Model
best_lda_model = model.best_estimator_
# Model Parameters
print("Best Model's Params: ", model.best_params_)
# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)
# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(data_vectorized))

Best Model's Params:  {'learning_decay': 0.4, 'n_components': 5}
Best Log Likelihood Score:  -638.4655157377955
Model Perplexity:  37.60799500301649


In [ ]:
# Create Document — Topic Matrix
lda_output = best_lda_model.transform(data_vectorized)
# column names
topicnames = ["Topic"+ str(i) for i in range(best_lda_model.n_components)]
# index names
docnames = ["Doc" + str(i) for i in range(len(custom_data))]
# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)
# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic["dominant_topic"] = dominant_topic
# Styling
def color_green(val):
 color = "green" if val > .1 else "black"
 return "color: {col}".format(col=color)
def make_bold(val):
 weight = 700 if val > .1 else 400
 return "font-weight: {weight}".format(weight=weight)
# Apply Style
df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
df_document_topics

,Topic0,Topic1,Topic2,Topic3,Topic4,dominant_topic
Doc0,0.040000,0.040000,0.040000,0.400000,0.480000,4
Doc1,0.340000,0.570000,0.030000,0.030000,0.030000,1
Doc2,0.600000,0.100000,0.100000,0.100000,0.100000,0
Doc3,0.730000,0.070000,0.070000,0.070000,0.070000,0
Doc4,0.050000,0.050000,0.800000,0.050000,0.050000,2
Doc5,0.530000,0.050000,0.050000,0.050000,0.310000,0
Doc6,0.120000,0.130000,0.030000,0.550000,0.170000,3
Doc7,0.040000,0.040000,0.210000,0.040000,0.670000,4
Doc8,0.100000,0.100000,0.100000,0.600000,0.100000,3
Doc9,0.370000,0.180000,0.080000,0.020000,0.360000,0


In [ ]:
# Topic-Keyword Matrix
df_topic_keywords = pd.DataFrame(best_lda_model.components_)
# Assign Column and Index
df_topic_keywords.columns = vectorizer.get_feature_names()
df_topic_keywords.index = topicnames
# View
df_topic_keywords.head()

,backlog,basis,business,chain,component,customer,demand,enterprise,good,growth,...,solid,strong,supply,team,term,thing,time,way,week,year
Topic0,17.953126,4.370770,4.532611,1.229756,0.200245,0.201941,2.394677,0.202740,0.200625,26.040001,...,6.819866,2.231446,0.201048,0.200024,1.254635,0.203938,0.200976,0.201943,8.296933,13.775827
Topic1,0.202176,0.202105,9.364425,0.200484,0.200112,4.771128,18.712452,0.345982,5.558917,0.898184,...,3.577977,8.105083,0.200596,0.201111,0.200390,0.271031,0.286196,7.199526,0.200025,2.530601
Topic2,0.200023,10.023756,4.205231,0.202442,9.574689,24.613842,0.200980,15.049877,0.212499,0.213123,...,0.200811,1.255650,1.404049,5.684033,2.789821,9.782246,0.200706,0.202157,2.451945,1.480756
Topic3,0.201612,0.200904,0.200399,10.166998,8.202029,0.202287,1.248421,0.201308,0.201426,4.813188,...,0.200484,1.205411,28.208632,2.836925,0.204673,4.230462,19.106194,5.166622,3.848362,3.305443
Topic4,0.443063,0.202464,2.697334,0.200320,1.822925,0.210803,0.443470,0.200094,5.826533,5.035504,...,0.200861,0.202410,0.985676,5.077908,12.550481,5.512322,1.205928,0.229751,0.202735,33.907374


In [ ]:
# Show top n keywords for each topic
def show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=5):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords
topic_keywords = show_topics(vectorizer=vectorizer, lda_model=best_lda_model, n_words=5)
# Topic - Keywords Dataframe
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords

,Word 0,Word 1,Word 2,Word 3,Word 4
Topic 0,revenue,growth,product,backlog,software
Topic 1,question,demand,business,strong,way
Topic 2,customer,enterprise,product,basis,thing
Topic 3,quarter,supply,time,chain,component
Topic 4,year,guidance,term,quarter,good


In [ ]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(best_lda_model, data_vectorized, vectorizer)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.201933 -0.090046       1        1  30.480209
3     -0.060921  0.227056       2        1  20.155355
2      0.102239 -0.073549       3        1  18.007547
4      0.006116  0.092806       4        1  17.886652
1     -0.249368 -0.156266       5        1  13.470237, topic_info=        Term       Freq      Total Category  logprob  loglift
31      year  54.000000  54.000000  Default  30.0000  30.0000
19  question  21.000000  21.000000  Default  29.0000  29.0000
5   customer  28.000000  28.000000  Default  28.0000  28.0000
20   revenue  39.000000  39.000000  Default  27.0000  27.0000
6     demand  21.000000  21.000000  Default  26.0000  26.0000
..       ...        ...        ...      ...      ...      ...
15     point   0.192815  21.366744   Topic5  -6.1575  -2.7032
11    impact   0.191877  23.075092   Topic5  -6.1624  -2.7850
24    supply   0.191686  27.715794   Topic5  -6.1634  -2.9692
20   revenue   0.191594  39.648512   Topic5  -6.1639  -3.3277
17   product   0.191589  33.635468   Topic5  -6.1639  -3.1633

[190 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
0         1  0.921570   backlog
1         1  0.279727     basis
1         3  0.629385     basis
2         1  0.197356  business
2         3  0.197356  business
...     ...       ...       ...
31        1  0.236879      year
31        2  0.054664      year
31        3  0.018221      year
31        4  0.637752      year
31        5  0.036443      year

[87 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 4, 3, 5, 2])

## v) LDA Analysis on ThermoFisher Dataset

In [ ]:
with open("Thermo_Fisher_transcript.txt","r") as f:
  string = f.read().split("\n")
  clean_string = []
  for s in string:
    if s.isspace() or s == "":
      pass
    else:
      clean_string.append(s)

cols = ["name","position"]
name_df = pd.read_csv("Thermo_Fisher_names.txt",sep="\t",names=cols)

chat_list = []
temp_name = ""
temp_dict = {}
user_names = list(name_df.name)+["Operator"]
for i in clean_string:
  if i in user_names and i!="":
    temp_name = i
    if i not in temp_dict:  
      temp_dict[i]= []
  else:
    if len(i)>20:
      temp_dict[temp_name].append(i)

In [ ]:
temp_dict["Marc Casper"]

['Thank you, Raf. Good morning, everyone and thanks for joining us today for our second quarter call.',
 'As you saw in our press release, we had another excellent quarter. We delivered outstanding financial performance.',
 'Our core business is performing very well, that strength is broad-based across all our businesses.',
 'As I reflect on the quarter, I am very proud of the team’s great execution and the resulting share gain we saw across our business.',
 'Our ongoing success is propelled by our proven growth strategy and our PPI Business System, which is a differentiator for us and enables operational excellence within the company.',
 'You will see this in our second quarter results and increased outlook for the year.',
 'So, let me first recap the financials.',
 'Our revenue in the quarter grew 18% year-over-year to $10.97 billion.',
 'Our adjusted operating income was $2.61 billion.',
 'Our adjusted operating margin in the second quarter was 23.7% and we delivered another quarter

In [ ]:
custom_data = temp_dict["Marc Casper"] + temp_dict["Rafael Tejada"]+temp_dict["Stephen Williamson"]

In [ ]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
data_words = list(sent_to_words(custom_data))
print(data_words[2])

['our', 'core', 'business', 'is', 'performing', 'very', 'well', 'that', 'strength', 'is', 'broad', 'based', 'across', 'all', 'our', 'businesses']


In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ']): #'NOUN', 'ADJ', 'VERB', 'ADV'
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out
    
data_lemmatized = lemmatization(data_words, allowed_postags=["NOUN","ADJ"]) #select noun and verb
print(data_lemmatized[:2])

['good morning thank today second quarter', 'press release excellent quarter outstanding financial performance']


In [ ]:
from nltk.stem import WordNetLemmatizer 
from nltk import word_tokenize      
class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, articles):
        return [self.wnl.lemmatize(t) for t in word_tokenize(articles)]

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
vectorizer_tfidf = TfidfVectorizer()
vectorizer = CountVectorizer(analyzer='word',min_df=10,stop_words='english', lowercase=True,token_pattern='[a-zA-Z0-9]{3,}'  )
data_vectorized = vectorizer.fit_transform(data_lemmatized)
print(data_vectorized)

  (0, 7)	1
  (0, 33)	1
  (0, 27)	1
  (0, 22)	1
  (1, 22)	1
  (1, 6)	1
  (1, 19)	1
  (2, 3)	1
  (2, 1)	2
  (3, 22)	1
  (3, 1)	1
  (3, 30)	1
  (3, 8)	1
  (4, 1)	1
  (4, 9)	1
  (4, 28)	1
  (4, 2)	1
  (5, 27)	1
  (5, 22)	1
  (5, 18)	1
  (5, 35)	1
  (7, 22)	1
  (7, 35)	2
  (7, 26)	1
  (9, 27)	1
  :	:
  (170, 11)	1
  (171, 35)	4
  (171, 24)	1
  (172, 33)	1
  (172, 23)	1
  (173, 35)	1
  (174, 35)	1
  (174, 5)	1
  (174, 34)	1
  (177, 27)	1
  (177, 35)	1
  (177, 16)	1
  (177, 11)	1
  (178, 7)	1
  (178, 30)	1
  (179, 15)	2
  (180, 22)	2
  (180, 3)	1
  (180, 1)	1
  (180, 35)	4
  (180, 15)	2
  (180, 32)	1
  (180, 31)	1
  (180, 20)	1
  (180, 0)	2


In [ ]:
# Build LDA Model
lda_model = LatentDirichletAllocation(n_components=10,               # Number of topics
                                      max_iter=15,               
# Max learning iterations
                                      learning_method='batch',   
                                      random_state=100,          
# Random state
                                      batch_size=128,            
# n docs in each learning iter
                                      evaluate_every = -1,       
# compute perplexity every n iters, default: Don't
                                      n_jobs = -1,               
# Use all available CPUs
                                     )
lda_output = lda_model.fit_transform(data_vectorized)
print(lda_output)

[[0.02       0.02000093 0.02       ... 0.02       0.21999933 0.41999309]
 [0.025      0.02500274 0.51651529 ... 0.025      0.025      0.28347726]
 [0.77498721 0.02500185 0.02500137 ... 0.025      0.02500073 0.025     ]
 ...
 [0.03335385 0.03333333 0.03334214 ... 0.03333333 0.69996701 0.03333333]
 [0.03333333 0.69999225 0.03333333 ... 0.03333333 0.03333333 0.03333871]
 [0.00625154 0.94373469 0.00625015 ... 0.00625    0.00625008 0.00625283]]


In [ ]:
# Log Likelyhood: Higher the better
print("Log Likelihood: ", lda_model.score(data_vectorized))
# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
print("Perplexity: ", lda_model.perplexity(data_vectorized))
# See model parameters
pprint(lda_model.get_params())

Log Likelihood:  -3255.086854940877
Perplexity:  39.56972251530781
{'batch_size': 128,
 'doc_topic_prior': None,
 'evaluate_every': -1,
 'learning_decay': 0.7,
 'learning_method': 'batch',
 'learning_offset': 10.0,
 'max_doc_update_iter': 100,
 'max_iter': 15,
 'mean_change_tol': 0.001,
 'n_components': 10,
 'n_jobs': -1,
 'perp_tol': 0.1,
 'random_state': 100,
 'topic_word_prior': None,
 'total_samples': 1000000.0,
 'verbose': 0}


In [ ]:
# Define Search Param
search_params = {'n_components': [ 5, 10, 15, 20, 25, 30], 'learning_decay': [.4, .5, .7, .9]}
# Init the Model
lda = LatentDirichletAllocation(max_iter=5, learning_method='batch', learning_offset=50.,random_state=0)
# Init Grid Search Class
model = GridSearchCV(lda, param_grid=search_params)
# Do the Grid Search
model.fit(data_vectorized)
GridSearchCV(cv=None, error_score='raise',
       estimator=LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7, learning_method=None,
             learning_offset=10.0, max_doc_update_iter=100, max_iter=15,
             mean_change_tol=0.001, n_components=None, n_jobs=1,
      perp_tol=0.1, random_state=None,
             topic_word_prior=None, total_samples=1000000.0, verbose=0), n_jobs=1,
       param_grid={'n_components': [5,10, 15, 20, 25, 30], 'learning_decay': [0.5, 0.7, 0.9]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

GridSearchCV(error_score='raise',
             estimator=LatentDirichletAllocation(learning_method=None,
                                                 max_iter=15, n_components=None,
                                                 n_jobs=1),
             n_jobs=1,
             param_grid={'learning_decay': [0.5, 0.7, 0.9],
                         'n_components': [5, 10, 15, 20, 25, 30]},
             return_train_score='warn')

In [ ]:
# Best Model
best_lda_model = model.best_estimator_
# Model Parameters
print("Best Model's Params: ", model.best_params_)
# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)
# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(data_vectorized))

Best Model's Params:  {'learning_decay': 0.4, 'n_components': 5}
Best Log Likelihood Score:  -823.9179434812864
Model Perplexity:  37.214267030551134


In [ ]:
# Create Document — Topic Matrix
lda_output = best_lda_model.transform(data_vectorized)
# column names
topicnames = ["Topic"+ str(i) for i in range(best_lda_model.n_components)]
# index names
docnames = ["Doc" + str(i) for i in range(len(custom_data))]
# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)
# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic["dominant_topic"] = dominant_topic
# Styling
def color_green(val):
 color = "green" if val > .1 else "black"
 return "color: {col}".format(col=color)
def make_bold(val):
 weight = 700 if val > .1 else 400
 return "font-weight: {weight}".format(weight=weight)
# Apply Style
df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
df_document_topics

,Topic0,Topic1,Topic2,Topic3,Topic4,dominant_topic
Doc0,0.040000,0.040000,0.530000,0.040000,0.340000,2
Doc1,0.050000,0.050000,0.800000,0.050000,0.050000,2
Doc2,0.470000,0.380000,0.050000,0.050000,0.050000,0
Doc3,0.410000,0.040000,0.470000,0.040000,0.040000,2
Doc4,0.840000,0.040000,0.040000,0.040000,0.040000,0
Doc5,0.040000,0.310000,0.040000,0.040000,0.570000,4
Doc6,0.200000,0.200000,0.200000,0.200000,0.200000,0
Doc7,0.040000,0.040000,0.040000,0.040000,0.840000,4
Doc8,0.200000,0.200000,0.200000,0.200000,0.200000,0
Doc9,0.030000,0.030000,0.120000,0.030000,0.790000,4


In [ ]:
# Topic-Keyword Matrix
df_topic_keywords = pd.DataFrame(best_lda_model.components_)
# Assign Column and Index
df_topic_keywords.columns = vectorizer.get_feature_names()
df_topic_keywords.index = topicnames
# View
df_topic_keywords.head()

,basis,business,company,core,covid,customer,excellent,good,great,growth,...,revenue,second,strategy,strong,team,term,testing,thank,time,year
Topic0,0.200119,27.028810,5.153655,0.201620,0.200783,25.217164,0.200293,3.077765,7.149619,18.959281,...,0.200871,0.200028,12.105763,2.656797,5.248133,5.669381,0.200726,0.200188,5.918424,0.202613
Topic1,0.200232,9.444241,0.202002,21.899198,13.507031,2.871685,0.203396,0.417736,0.203278,11.501370,...,22.085719,4.333741,0.740472,14.291711,1.582813,3.545726,12.201423,0.200688,0.317191,11.773371
Topic2,0.200080,12.686340,0.200017,3.604250,2.864939,1.509212,12.190531,17.409781,0.403547,4.632387,...,0.204221,0.209095,0.518651,0.201054,11.808676,0.200040,2.924790,12.065083,0.201195,0.200965
Topic3,0.202836,2.263899,15.243569,0.200863,0.200847,0.201681,0.200399,7.894190,0.203497,4.388193,...,0.200642,0.201884,0.386561,17.653970,2.160266,2.460450,0.200357,1.333803,5.329449,0.201157
Topic4,13.196734,21.576709,0.200757,7.094069,0.226399,0.200258,0.205381,0.200528,8.040059,11.518770,...,28.308547,12.055252,1.248553,14.196468,0.200112,4.124403,4.472704,0.200237,1.233741,70.621894


In [ ]:
# Show top n keywords for each topic
def show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=5):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords
topic_keywords = show_topics(vectorizer=vectorizer, lda_model=best_lda_model, n_words=5)
# Topic - Keywords Dataframe
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords

,Word 0,Word 1,Word 2,Word 3,Word 4
Topic 0,business,customer,growth,strategy,large
Topic 1,revenue,core,outlook,strong,covid
Topic 2,good,performance,business,excellent,question
Topic 3,market,strong,company,good,time
Topic 4,year,quarter,revenue,business,guidance


In [ ]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(best_lda_model, data_vectorized, vectorizer)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.234125 -0.020605       1        1  36.984894
1      0.174794  0.003631       2        1  23.257373
0     -0.089856  0.170839       3        1  16.551123
2     -0.140202 -0.206415       4        1  13.189967
3     -0.178861  0.052551       5        1  10.016643, topic_info=        Term       Freq      Total Category  logprob  loglift
29    strong  48.000000  48.000000  Default  30.0000  30.0000
16    market  24.000000  24.000000  Default  29.0000  29.0000
5   customer  29.000000  29.000000  Default  28.0000  28.0000
35      year  77.000000  77.000000  Default  27.0000  27.0000
2    company  21.000000  21.000000  Default  26.0000  26.0000
..       ...        ...        ...      ...      ...      ...
10  guidance   0.204771  29.640018   Topic5  -6.0705  -2.6741
22   quarter   0.208347  49.148893   Topic5  -6.0532  -3.1625
5   customer   0.202393  29.895911   Topic5  -6.0822  -2.6943
35      year   0.201867  77.180834   Topic5  -6.0848  -3.6454
3       core   0.201571  33.016320   Topic5  -6.0863  -2.7977

[201 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
0         1  0.937132     basis
1         1  0.284949  business
1         2  0.142474  business
1         3  0.384681  business
1         4  0.156722  business
...     ...       ...       ...
34        1  0.077602      time
34        3  0.465613      time
34        5  0.388011      time
35        1  0.829221      year
35        2  0.155479      year

[95 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 2, 1, 3, 4])